# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-106b3c299a-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Tv series Without a Trace explorative search") 


Consider the following exploratory scenario:


> we are interested in the TV series "Without a Trace" and we want to investigate the main aspects related to the actors and directors involved in the production, know the numerber of seasons and check what are the episodes which got the higher success/impact.


## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P4969`    | derivative work      | predicate |
| `wd:Q826477` | Without a Trace    | node |
| `wd:Q733960` | Cold Case       | node |



Also consider

```
wd:Q826477 ?p ?obj .
```

is the BGP to retrieve all **properties of Without a Trace**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.



The workload should

1. Identify the BGP for television series

2. Return the number of seasons and episodes per season of the tv series (the result set must be triples of season IRI, label and #episodes).

3. Get the number of episodes in which the cast members played a role. Who are the most present actors? (the result set must be a list of triples actor/actress IRI, label and #episodes)

4. Check who is the actor who acted in more films while working on "Without a Trace" (the result set must be a list of triples actor/actress IRI, label and #films).

5. Compare Without a Trace with the tv series "Cold Case" in terms of number of seasons, episods and cast members (the result set must be two elements -one for each tv series- of tv series IRI, label, #seasons, #episodes and #cast members).

6. Return the actors who are members of the cast of Without a Trace have [Kavin Bacon number](https://en.wikipedia.org/wiki/Six_Degrees_of_Kevin_Bacon#:~:text=Kevin%20Bacon%20himself%20has%20a,Bacon%20number%20is%20N%2B1.) equal to 2 (the result set must be a list of couples actor/actress IRI and label).

7. Consider the actors who are members of the cast of Cold Case. Amongst the tv series which these actors acted return only those which received more than 2 awards (the result set must be triples of tv series IRI, label, #awards won).

## Task 1
Identify the BGP for television series

In [2]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q826477 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('name', 'number of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1267'), ('name', 'AlloCiné series ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('name', 'title')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P1562'), ('name', 'AllMovie title ID')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P170'), ('name', 'creator')]
[('p', 'http://www.wikidata.org/prop/direct/P1712'), ('name', 'Metacritic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1811'), ('name', 'list of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P2047'), ('name', 'duration')]
[('p', 'http://www.wikidata.org/prop/direct/P2

Final query for this task

In [243]:
queryString = """
SELECT DISTINCT ?instanceOf ?name
WHERE {
   wd:Q826477 wdt:P31 ?instanceOf .
    ?instanceOf sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('instanceOf', 'http://www.wikidata.org/entity/Q5398426'), ('name', 'television series')]
1


## Task 2
Return the number of seasons and episodes per season of the tv series (the result set must be triples of season IRI, label and #episodes).

In [8]:
# get all subjects that are instance of televisions series
queryString = """
SELECT DISTINCT ?tvSerie
WHERE {
   ?tvSerie wdt:P31 wd:Q5398426 .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('tvSerie', 'http://www.wikidata.org/entity/Q1154217')]
[('tvSerie', 'http://www.wikidata.org/entity/Q691528')]
[('tvSerie', 'http://www.wikidata.org/entity/Q8187904')]
[('tvSerie', 'http://www.wikidata.org/entity/Q8193809')]
[('tvSerie', 'http://www.wikidata.org/entity/Q33323')]
[('tvSerie', 'http://www.wikidata.org/entity/Q500769')]
[('tvSerie', 'http://www.wikidata.org/entity/Q8350955')]
[('tvSerie', 'http://www.wikidata.org/entity/Q8354249')]
[('tvSerie', 'http://www.wikidata.org/entity/Q1145054')]
[('tvSerie', 'http://www.wikidata.org/entity/Q344081')]
[('tvSerie', 'http://www.wikidata.org/entity/Q13647845')]
[('tvSerie', 'http://www.wikidata.org/entity/Q13725650')]
[('tvSerie', 'http://www.wikidata.org/entity/Q13725885')]
[('tvSerie', 'http://www.wikidata.org/entity/Q708661')]
[('tvSerie', 'http://www.wikidata.org/entity/Q15099450')]
[('tvSerie', 'http://www.wikidata.org/entity/Q15476155')]
[('tvSerie', 'http://www.wikidata.org/entity/Q15933273')]
[('tvSerie', 'http://ww

In [13]:
queryString = """
SELECT DISTINCT ?prop ?name
WHERE {
   wd:Q826477 ?prop ?value .
   ?prop sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1113'), ('name', 'number of episodes')]
[('prop', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1267'), ('name', 'AlloCiné series ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('prop', 'http://www.wikidata.org/prop/direct/P1476'), ('name', 'title')]
[('prop', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('prop', 'http://www.wikidata.org/prop/direct/P1562'), ('name', 'AllMovie title ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]
[('prop', 'http://www.wikidata.org/prop/direct/P170'), ('name', 'creator')]
[('prop', 'http://www.wikidata.org/prop/direct/P1712'), ('name', 'Metacritic ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1811'), ('name', 'list of episodes')]
[('prop', 'http://www.wikidata.org/prop/direct/P2047'), ('name', 'duration')]
[('prop', 

In [14]:
# Get the number of seasons of a given TV series
queryString = """
SELECT DISTINCT ?seasons
WHERE {
   wd:Q826477 wdt:P2437 ?seasons .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('seasons', '7')]
1


In [15]:
# Get the list of episodes of a given TV series
queryString = """
SELECT DISTINCT ?episodes
WHERE {
   wd:Q826477 wdt:P1811 ?episodes .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('episodes', 'http://www.wikidata.org/entity/Q282789')]
1


In [16]:
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q282789 ?p ?v .
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P360'), ('name', 'is a list of')]
2


In [19]:
queryString = """
SELECT DISTINCT ?name
WHERE {
   wd:Q282789 wdt:P31 ?instanceOf .
   ?instanceOf sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('name', 'Wikimedia list article')]
1


In [21]:
queryString = """
SELECT DISTINCT ?instanceOf ?name
WHERE {
   wd:Q282789 wdt:P360 ?instanceOf .
   ?instanceOf sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('instanceOf', 'http://www.wikidata.org/entity/Q1983062'), ('name', 'episode')]
[('instanceOf', 'http://www.wikidata.org/entity/Q21191270'), ('name', 'television series episode')]
2


In [23]:
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q21191270 ?p ?v .
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('name', 'has quality')]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('name', 'equivalent class')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('name', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('name', 'Quora topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P6104'), ('name', 'maintained by WikiProject')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('name', 'Freebase ID')]
[('p', 'http://www.wikidata.org/prop/direct/P8408'), ('name', 'KBpedia ID')]
[('p', 'http://w

In [25]:
queryString = """
SELECT DISTINCT ?partOf
WHERE {
   wd:Q21191270 wdt:P361 ?partOf .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('partOf', 'http://www.wikidata.org/entity/Q5398426')]
[('partOf', 'http://www.wikidata.org/entity/Q3464665')]
2


In [27]:
queryString = """
SELECT DISTINCT ?partOf ?name
WHERE {
   wd:Q21191270 wdt:P361 ?partOf .
   ?partOf sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('partOf', 'http://www.wikidata.org/entity/Q5398426'), ('name', 'television series')]
[('partOf', 'http://www.wikidata.org/entity/Q3464665'), ('name', 'television series season')]
2


In [28]:
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q3464665 ?p ?v .
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('name', 'has quality')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('name', 'Wikidata property')]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('name', 'equivalent class')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('name', 'has list')]
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('name', 'has parts of the class')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P5869'), ('name', 'model item')]
[('p', 'http://www.wikidata.org/prop/direct/P6104'), ('name', 'maintained by WikiProj

In [29]:
# Get all TV series seasons
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P31 wd:Q3464665 .
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q1114794'), ('name', "Grey's Anatomy, season 1")]
[('s', 'http://www.wikidata.org/entity/Q17090112'), ('name', 'Melodifestivalen 2004')]
[('s', 'http://www.wikidata.org/entity/Q256273'), ('name', 'Melodifestivalen 2009')]
[('s', 'http://www.wikidata.org/entity/Q3061434'), ('name', 'Law & Order: Special Victims Unit, season 4')]
[('s', 'http://www.wikidata.org/entity/Q15726792'), ('name', 'Face Off, season 6')]
[('s', 'http://www.wikidata.org/entity/Q15727770'), ('name', 'Grande Fratello, season 13')]
[('s', 'http://www.wikidata.org/entity/Q15732920'), ('name', 'Lost Girl, season 4')]
[('s', 'http://www.wikidata.org/entity/Q3729305'), ('name', 'My Little Pony: Friendship Is Magic, season 3')]
[('s', 'http://www.wikidata.org/entity/Q65167413'), ('name', 'Power Rangers Dino Super Charge')]
[('s', 'http://www.wikidata.org/entity/Q96472097'), ('name', 'Power Rangers Dino Charge, season 1')]
[('s', 'http://www.wikidata.org/entity/Q12294356'), ('

In [30]:
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   ?Q1114794 ?p ?v .
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3296'), ('name', 'DRÚSOP ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2377'), ('name', 'MediaWiki hooks used')]
[('p', 'http://www.wikidata.org/prop/direct/P8660'), ('name', 'Native Plants Database ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1161'), ('name', 'Z39.5 abbreviation')]
[('p', 'http://www.wikidata.org/prop/direct/P1210'), ('name', 'supercharger')]
[('p', 'http://www.wikidata.org/prop/direct/P3431'), ('name', 'Publons publication ID')]
[('p', 'http://www.wikidata.org/prop/direct/P6123'), ('name', 'B3Kat dataset ID')]
[('p', 'http://www.wikidata.org/prop/direct/P7983'), ('name', 'ACM Journal ID')]
[('p', 'http://www.wikidata.org/prop/direct/P862'), ('name', 'Operational Requirement of the UK Air Ministry')]
[('p', 'http://www.wikidata.org/prop/direct/P9234'), ('name', 'supplement to')]
[('p', 'http://www.wikidata.org/prop/direct/P3903'), ('name', 'column')]
[('p', 'http://www.wikidata.org/prop/direct/P7494'), ('

In [31]:
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q1114794 ?p ?v .
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('name', 'number of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P179'), ('name', 'part of the series')]
[('p', 'http://www.wikidata.org/prop/direct/P2747'), ('name', 'Filmiroda rating')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P364'), ('name', 'original language of film or TV show')]
[('p', 'http://www.wikidata.org/prop/direct/P437'), ('name', 'distribution format')]
[('p', 'http://www.wikidata.org/prop/direct/P449'), ('name', 'original broadcaster')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part')]
[('p', 'http://www.wikidata.org/prop/direct/P580'), ('name', 'start time')]
[('p', 'http://www.wikidata.org/prop/direct/P582'), ('name', 'end time')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('name', 'Freebase ID')]
[('p'

In [32]:
queryString = """
SELECT DISTINCT ?partOf ?name
WHERE {
   wd:Q1114794 wdt:P179 ?partOf .
   ?partOf sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('partOf', 'http://www.wikidata.org/entity/Q438406'), ('name', "Grey's Anatomy")]
1


In [235]:
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q438406 ?p ?v .
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P5829'), ('name', 'Television Academy Foundation show ID')]
[('p', 'http://www.wikidata.org/prop/direct/P5255'), ('name', 'FilmTv.it TV series ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('name', 'number of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1267'), ('name', 'AlloCiné series ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('name', 'title')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P1562'), ('name', 'AllMovie title ID')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'ca

In [34]:
queryString = """
SELECT DISTINCT ?v ?name
WHERE {
   wd:Q438406 wdt:P31 ?v .
   ?v sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('v', 'http://www.wikidata.org/entity/Q5398426'), ('name', 'television series')]
1


In [35]:
# Get all the TV series
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P31 wd:Q5398426 .
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q1154217'), ('name', 'Ironfist Chinmi')]
[('s', 'http://www.wikidata.org/entity/Q691528'), ('name', 'Wallace and Gromit')]
[('s', 'http://www.wikidata.org/entity/Q8193809'), ('name', 'Shomuni')]
[('s', 'http://www.wikidata.org/entity/Q33323'), ('name', 'Everybody Hates Chris')]
[('s', 'http://www.wikidata.org/entity/Q500769'), ('name', 'Freaks and Geeks')]
[('s', 'http://www.wikidata.org/entity/Q8350955'), ('name', 'Cordel Encantado')]
[('s', 'http://www.wikidata.org/entity/Q1145054'), ('name', 'The NBC Mystery Movie')]
[('s', 'http://www.wikidata.org/entity/Q344081'), ('name', 'Recess')]
[('s', 'http://www.wikidata.org/entity/Q13647845'), ('name', 'Grijpstra & De Gier')]
[('s', 'http://www.wikidata.org/entity/Q708661'), ('name', 'Lindenstraße')]
[('s', 'http://www.wikidata.org/entity/Q15099450'), ('name', 'Prime Minister and I')]
[('s', 'http://www.wikidata.org/entity/Q15476155'), ('name', 'The Full Sun')]
[('s', 'http://www.wikidata.org/

In [36]:
# Get all the seasons and the number of episodes per season for all tv series
queryString = """
SELECT DISTINCT ?season ?name ?episodes
WHERE {
   ?season wdt:P31 wd:Q3464665 .
   ?season sc:name ?name.
   ?season wdt:P1113 ?episodes
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('season', 'http://www.wikidata.org/entity/Q1114794'), ('name', "Grey's Anatomy, season 1"), ('episodes', '9')]
[('season', 'http://www.wikidata.org/entity/Q3061434'), ('name', 'Law & Order: Special Victims Unit, season 4'), ('episodes', '25')]
[('season', 'http://www.wikidata.org/entity/Q15726792'), ('name', 'Face Off, season 6'), ('episodes', '15')]
[('season', 'http://www.wikidata.org/entity/Q15732920'), ('name', 'Lost Girl, season 4'), ('episodes', '13')]
[('season', 'http://www.wikidata.org/entity/Q3729305'), ('name', 'My Little Pony: Friendship Is Magic, season 3'), ('episodes', '13')]
[('season', 'http://www.wikidata.org/entity/Q12294356'), ('name', 'Grimm, season 3'), ('episodes', '22')]
[('season', 'http://www.wikidata.org/entity/Q15844444'), ('name', 'Schloss Einstein, season 2'), ('episodes', '40')]
[('season', 'http://www.wikidata.org/entity/Q3729003'), ('name', 'Law & Order: Criminal Intent, season 9'), ('episodes', '16')]
[('season', 'http://www.wikidata.org/enti

Final query for this task

Assuming that we are interested only in the seasons and episodes per season of "Without a Trace". If the query was about ALL tv series then execute the query above without the limit clause.

In [ ]:
queryString = """
SELECT DISTINCT ?season ?name ?episodes
WHERE {
   ?season wdt:P179 wd:Q826477 . 
   ?season sc:name ?name.
   ?season wdt:P1113 ?episodes
}
"""

print("Results")
x=run_query(queryString)

Results
[('season', 'http://www.wikidata.org/entity/Q1120248'), ('name', 'Without a Trace, season 1'), ('episodes', '23')]
[('season', 'http://www.wikidata.org/entity/Q3729810'), ('name', 'Without a Trace, season 4'), ('episodes', '24')]
[('season', 'http://www.wikidata.org/entity/Q3729811'), ('name', 'Without a Trace, season 5'), ('episodes', '24')]
[('season', 'http://www.wikidata.org/entity/Q3729812'), ('name', 'Without a Trace, season 2'), ('episodes', '24')]
[('season', 'http://www.wikidata.org/entity/Q3729815'), ('name', 'Without a Trace, season 6'), ('episodes', '18')]
[('season', 'http://www.wikidata.org/entity/Q3729816'), ('name', 'Without a Trace, season 7'), ('episodes', '24')]
[('season', 'http://www.wikidata.org/entity/Q3729817'), ('name', 'Without a Trace, season 3'), ('episodes', '23')]
7


## Task 3
Get the number of episodes in which the cast members played a role. Who are the most present actors? (the result set must be a list of triples actor/actress IRI, label and #episodes)

Given the context describe at the head of the notebook I assume that we are interested only in episodes of "Without a Trace".

In [38]:
# Get all the cast members for 'Without a Trace'
queryString = """
SELECT DISTINCT ?cast ?name
WHERE {
   wd:Q826477 wdt:P161 ?cast .
   ?cast sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('cast', 'http://www.wikidata.org/entity/Q308124'), ('name', 'Anthony LaPaglia')]
[('cast', 'http://www.wikidata.org/entity/Q235075'), ('name', 'Poppy Montgomery')]
2


In [39]:
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q308124 ?p ?v .
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('name', 'NUKAT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1220'), ('name', 'Internet Broadway Database person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1266'), ('name', 'AlloCiné person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('name', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
[('p', 'http://www.wikidata.org/prop/direct/P1695'), ('name', 'NLP ID (unique)')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P19'),

In [42]:
# Get data about season one of 'Without a Trace'
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q1120248 ?p ?v .
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('name', 'number of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1712'), ('name', 'Metacritic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P179'), ('name', 'part of the series')]
[('p', 'http://www.wikidata.org/prop/direct/P2529'), ('name', 'ČSFD film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('name', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2638'), ('name', 'TV.com ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2704'), ('name', 'EIDR content ID')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3302'), ('name', 'Open Media Database film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P364'), ('name', 'original language of film or TV show')]
[('p', 'http://www.wikidata.org/prop/direct/P437'), ('name', 'distribut

In [46]:
# Get info about episodes season one of 'Without a Trace'
queryString = """
SELECT DISTINCT ?v ?name
WHERE {
   wd:Q1120248 wdt:P527 ?v .
   ?v sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('v', 'http://www.wikidata.org/entity/Q52667904'), ('name', 'Maple Street')]
[('v', 'http://www.wikidata.org/entity/Q52667908'), ('name', 'Underground Railroad')]
[('v', 'http://www.wikidata.org/entity/Q52667910'), ('name', 'Hang On to Me')]
[('v', 'http://www.wikidata.org/entity/Q39069555'), ('name', 'Pilot')]
[('v', 'http://www.wikidata.org/entity/Q52667878'), ('name', 'Birthday Boy')]
[('v', 'http://www.wikidata.org/entity/Q52667894'), ('name', 'Snatch Back')]
[('v', 'http://www.wikidata.org/entity/Q52667898'), ('name', 'Little Big Man')]
[('v', 'http://www.wikidata.org/entity/Q52667900'), ('name', 'In Extremis')]
[('v', 'http://www.wikidata.org/entity/Q52667911'), ('name', 'The Friendly Skies')]
[('v', 'http://www.wikidata.org/entity/Q52667916'), ('name', 'There Goes the Bride')]
[('v', 'http://www.wikidata.org/entity/Q52667920'), ('name', 'Clare de Lune')]
[('v', 'http://www.wikidata.org/entity/Q52667923'), ('name', 'Kam Li')]
[('v', 'http://www.wikidata.org/entity/Q52667

In [47]:
# Get info about episode 'Pilot' of season one of 'Without a Trace'
queryString = """
SELECT DISTINCT ?p ?prop ?v ?val
WHERE {
   wd:Q39069555 ?p ?v .
   ?p sc:name ?prop.
   ?v sc:name ?val.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('prop', 'cast member'), ('v', 'http://www.wikidata.org/entity/Q139341'), ('val', 'Zach Grenier')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('prop', 'cast member'), ('v', 'http://www.wikidata.org/entity/Q430872'), ('val', 'Marianne Jean-Baptiste')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('prop', 'cast member'), ('v', 'http://www.wikidata.org/entity/Q308124'), ('val', 'Anthony LaPaglia')]
[('p', 'http://www.wikidata.org/prop/direct/P57'), ('prop', 'director'), ('v', 'http://www.wikidata.org/entity/Q1279660'), ('val', 'David Nutter')]
[('p', 'http://www.wikidata.org/prop/direct/P364'), ('prop', 'original language of film or TV show'), ('v', 'http://www.wikidata.org/entity/Q1860'), ('val', 'English')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('prop', 'instance of'), ('v', 'http://www.wikidata.org/entity/Q21191270'), ('val', 'television series episode')]
[('p', 'http://www.wikidata.org/prop/direct

In [48]:
# Get info about episode 'Pilot' of season one of 'Without a Trace'
queryString = """
SELECT DISTINCT ?p ?prop
WHERE {
   wd:Q39069555 ?p ?v .
   ?p sc:name ?prop.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('prop', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('prop', 'title')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('prop', 'followed by')]
[('p', 'http://www.wikidata.org/prop/direct/P1562'), ('prop', 'AllMovie title ID')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('prop', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P179'), ('prop', 'part of the series')]
[('p', 'http://www.wikidata.org/prop/direct/P2364'), ('prop', 'production code')]
[('p', 'http://www.wikidata.org/prop/direct/P2529'), ('prop', 'ČSFD film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2638'), ('prop', 'TV.com ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('prop', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2703'), ('prop', 'BFI National Archive work ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2704'), ('prop', 'EIDR content ID')]
[('p', 

In [49]:
# Get info about cast members of episode 'Pilot' of season one of 'Without a Trace'
queryString = """
SELECT DISTINCT ?v ?name
WHERE {
   wd:Q39069555 wdt:P161 ?v .
   ?v sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('v', 'http://www.wikidata.org/entity/Q139341'), ('name', 'Zach Grenier')]
[('v', 'http://www.wikidata.org/entity/Q430872'), ('name', 'Marianne Jean-Baptiste')]
[('v', 'http://www.wikidata.org/entity/Q308124'), ('name', 'Anthony LaPaglia')]
[('v', 'http://www.wikidata.org/entity/Q355163'), ('name', 'Bruce Davison')]
[('v', 'http://www.wikidata.org/entity/Q1125651'), ('name', 'Thom Barry')]
[('v', 'http://www.wikidata.org/entity/Q235075'), ('name', 'Poppy Montgomery')]
[('v', 'http://www.wikidata.org/entity/Q551608'), ('name', 'Enrique Murciano')]
[('v', 'http://www.wikidata.org/entity/Q503040'), ('name', 'Eric Close')]
[('v', 'http://www.wikidata.org/entity/Q462343'), ('name', 'Rosemary Forsyth')]
[('v', 'http://www.wikidata.org/entity/Q2923471'), ('name', 'Bradford Tatum')]
10


In [50]:
# Get info about cast members in all episodes of season one of 'Without a Trace'
queryString = """
SELECT DISTINCT ?castMember ?castMemberName
WHERE {
   wd:Q1120248 wdt:P527 ?episode .
   ?episode wdt:P161 ?castMember.
   ?castMember sc:name ?castMemberName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q373989'), ('castMemberName', 'David Paymer')]
[('castMember', 'http://www.wikidata.org/entity/Q139341'), ('castMemberName', 'Zach Grenier')]
[('castMember', 'http://www.wikidata.org/entity/Q430872'), ('castMemberName', 'Marianne Jean-Baptiste')]
[('castMember', 'http://www.wikidata.org/entity/Q238501'), ('castMemberName', 'Alex Veadov')]
[('castMember', 'http://www.wikidata.org/entity/Q308124'), ('castMemberName', 'Anthony LaPaglia')]
[('castMember', 'http://www.wikidata.org/entity/Q1721796'), ('castMemberName', 'Kaitlin Doubleday')]
[('castMember', 'http://www.wikidata.org/entity/Q355163'), ('castMemberName', 'Bruce Davison')]
[('castMember', 'http://www.wikidata.org/entity/Q22704603'), ('castMemberName', 'Lily Knight')]
[('castMember', 'http://www.wikidata.org/entity/Q445325'), ('castMemberName', 'Megan Gallagher')]
[('castMember', 'http://www.wikidata.org/entity/Q706513'), ('castMemberName', 'Charles S. Dutton')]
[('castMember

In [54]:
# Get info about cast members in all episodes of season one of 'Without a Trace'
queryString = """
SELECT ?castMember ?castMemberName (COUNT(?castMember) as ?episodes)
WHERE {
   wd:Q1120248 wdt:P527 ?episode .
   ?episode wdt:P161 ?castMember.
   ?castMember sc:name ?castMemberName.
}
GROUP BY ?castMember ?castMemberName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q308124'), ('castMemberName', 'Anthony LaPaglia'), ('episodes', '23')]
[('castMember', 'http://www.wikidata.org/entity/Q459638'), ('castMemberName', 'Stacy Edwards'), ('episodes', '1')]
[('castMember', 'http://www.wikidata.org/entity/Q706513'), ('castMemberName', 'Charles S. Dutton'), ('episodes', '1')]
[('castMember', 'http://www.wikidata.org/entity/Q5363239'), ('castMemberName', 'Elizabeth Morehead'), ('episodes', '1')]
[('castMember', 'http://www.wikidata.org/entity/Q1721796'), ('castMemberName', 'Kaitlin Doubleday'), ('episodes', '1')]
[('castMember', 'http://www.wikidata.org/entity/Q235075'), ('castMemberName', 'Poppy Montgomery'), ('episodes', '23')]
[('castMember', 'http://www.wikidata.org/entity/Q297128'), ('castMemberName', 'David Henrie'), ('episodes', '1')]
[('castMember', 'http://www.wikidata.org/entity/Q503040'), ('castMemberName', 'Eric Close'), ('episodes', '23')]
[('castMember', 'http://www.wikidata.org/entity/Q326

In [56]:
# Get info about cast members in all episodes of season one of 'Without a Trace'
queryString = """
SELECT ?castMember ?castMemberName
WHERE {
   wd:Q1120248 wdt:P527 ?episode .
   ?episode wdt:P161 ?castMember.
   ?castMember sc:name ?castMemberName.
}
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q373989'), ('castMemberName', 'David Paymer')]
[('castMember', 'http://www.wikidata.org/entity/Q139341'), ('castMemberName', 'Zach Grenier')]
[('castMember', 'http://www.wikidata.org/entity/Q430872'), ('castMemberName', 'Marianne Jean-Baptiste')]
[('castMember', 'http://www.wikidata.org/entity/Q430872'), ('castMemberName', 'Marianne Jean-Baptiste')]
[('castMember', 'http://www.wikidata.org/entity/Q430872'), ('castMemberName', 'Marianne Jean-Baptiste')]
[('castMember', 'http://www.wikidata.org/entity/Q430872'), ('castMemberName', 'Marianne Jean-Baptiste')]
[('castMember', 'http://www.wikidata.org/entity/Q430872'), ('castMemberName', 'Marianne Jean-Baptiste')]
[('castMember', 'http://www.wikidata.org/entity/Q430872'), ('castMemberName', 'Marianne Jean-Baptiste')]
[('castMember', 'http://www.wikidata.org/entity/Q430872'), ('castMemberName', 'Marianne Jean-Baptiste')]
[('castMember', 'http://www.wikidata.org/entity/Q430872'), ('castMe

In [58]:
# Get info about cast members in all episodes of season one of 'Without a Trace'
queryString = """
SELECT ?castMember ?castMemberName (COUNT(?castMember) as ?episodes)
WHERE {
   wd:Q1120248 wdt:P527 ?episode .
   ?episode wdt:P161 ?castMember.
   ?castMember sc:name ?castMemberName.
}
GROUP BY ?castMember ?castMemberName
ORDER BY DESC(?episodes)
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q308124'), ('castMemberName', 'Anthony LaPaglia'), ('episodes', '23')]
[('castMember', 'http://www.wikidata.org/entity/Q235075'), ('castMemberName', 'Poppy Montgomery'), ('episodes', '23')]
[('castMember', 'http://www.wikidata.org/entity/Q503040'), ('castMemberName', 'Eric Close'), ('episodes', '23')]
[('castMember', 'http://www.wikidata.org/entity/Q551608'), ('castMemberName', 'Enrique Murciano'), ('episodes', '23')]
[('castMember', 'http://www.wikidata.org/entity/Q430872'), ('castMemberName', 'Marianne Jean-Baptiste'), ('episodes', '23')]
[('castMember', 'http://www.wikidata.org/entity/Q459638'), ('castMemberName', 'Stacy Edwards'), ('episodes', '1')]
[('castMember', 'http://www.wikidata.org/entity/Q706513'), ('castMemberName', 'Charles S. Dutton'), ('episodes', '1')]
[('castMember', 'http://www.wikidata.org/entity/Q5363239'), ('castMemberName', 'Elizabeth Morehead'), ('episodes', '1')]
[('castMember', 'http://www.wikidata.org/e

In [62]:
# Get info about all the episodes in all the seasons of 'Without a Trace'
queryString = """
SELECT DISTINCT ?epi ?epiName 
WHERE {
   ?season wdt:P179 wd:Q826477 . #BGP to retrieve all the seasons
   ?season wdt:P527 ?epi .       #BGP to retrieve all the episodes of a season
   ?epi sc:name ?epiName.        #BGP to retrieve the name of the episodes
}
"""

print("Results")
x=run_query(queryString)

Results
[('epi', 'http://www.wikidata.org/entity/Q52667904'), ('epiName', 'Maple Street')]
[('epi', 'http://www.wikidata.org/entity/Q52667908'), ('epiName', 'Underground Railroad')]
[('epi', 'http://www.wikidata.org/entity/Q52667910'), ('epiName', 'Hang On to Me')]
[('epi', 'http://www.wikidata.org/entity/Q52667965'), ('epiName', 'Our Sons and Daughters')]
[('epi', 'http://www.wikidata.org/entity/Q52667968'), ('epiName', 'A Tree Falls')]
[('epi', 'http://www.wikidata.org/entity/Q52667970'), ('epiName', 'Trip Box')]
[('epi', 'http://www.wikidata.org/entity/Q52667973'), ('epiName', 'Moving On')]
[('epi', 'http://www.wikidata.org/entity/Q52667986'), ('epiName', 'Life Rules')]
[('epi', 'http://www.wikidata.org/entity/Q52667989'), ('epiName', 'The Line')]
[('epi', 'http://www.wikidata.org/entity/Q52667991'), ('epiName', 'Wannabe')]
[('epi', 'http://www.wikidata.org/entity/Q52667993'), ('epiName', 'Risen')]
[('epi', 'http://www.wikidata.org/entity/Q52667996'), ('epiName', 'Gung Ho')]
[('epi'

Final query for this task

In [245]:
# Get info about all the episodes in all the seasons of 'Without a Trace'
queryString = """
SELECT ?castMember ?castMemberName (COUNT(?castMember) as ?episodes) 
WHERE {
   ?season wdt:P179 wd:Q826477 .        #BGP to retrieve all the seasons
   ?season wdt:P527 ?episode .          #BGP to retrieve all the episodes of a season
   ?episode wdt:P161 ?castMember.       #BGP to retireve all cast members that participated in the episode
   ?castMember sc:name ?castMemberName.
}
GROUP BY ?castMember ?castMemberName
ORDER BY DESC(?episodes)
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q308124'), ('castMemberName', 'Anthony LaPaglia'), ('episodes', '160')]
[('castMember', 'http://www.wikidata.org/entity/Q235075'), ('castMemberName', 'Poppy Montgomery'), ('episodes', '160')]
[('castMember', 'http://www.wikidata.org/entity/Q503040'), ('castMemberName', 'Eric Close'), ('episodes', '160')]
[('castMember', 'http://www.wikidata.org/entity/Q551608'), ('castMemberName', 'Enrique Murciano'), ('episodes', '160')]
[('castMember', 'http://www.wikidata.org/entity/Q430872'), ('castMemberName', 'Marianne Jean-Baptiste'), ('episodes', '160')]
[('castMember', 'http://www.wikidata.org/entity/Q459638'), ('castMemberName', 'Stacy Edwards'), ('episodes', '1')]
[('castMember', 'http://www.wikidata.org/entity/Q706513'), ('castMemberName', 'Charles S. Dutton'), ('episodes', '1')]
[('castMember', 'http://www.wikidata.org/entity/Q5363239'), ('castMemberName', 'Elizabeth Morehead'), ('episodes', '1')]
[('castMember', 'http://www.wikidata.

## Task 4
Check who is the actor who acted in more films while working on "Without a Trace" (the result set must be a list of triples actor/actress IRI, label and #films).

In [64]:
# Get info about all the actors that participated in at least one episode of 'Without a Trace'
queryString = """
SELECT DISTINCT ?castMember ?castMemberName
WHERE {
   ?season wdt:P179 wd:Q826477 .        #BGP to retrieve all the seasons
   ?season wdt:P527 ?episode .          #BGP to retrieve all the episodes of a season
   ?episode wdt:P161 ?castMember.       #BGP to retireve all cast members that participated in the episode
   ?castMember sc:name ?castMemberName.
}
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q373989'), ('castMemberName', 'David Paymer')]
[('castMember', 'http://www.wikidata.org/entity/Q139341'), ('castMemberName', 'Zach Grenier')]
[('castMember', 'http://www.wikidata.org/entity/Q3265830'), ('castMemberName', 'Lucille Soong')]
[('castMember', 'http://www.wikidata.org/entity/Q430872'), ('castMemberName', 'Marianne Jean-Baptiste')]
[('castMember', 'http://www.wikidata.org/entity/Q238501'), ('castMemberName', 'Alex Veadov')]
[('castMember', 'http://www.wikidata.org/entity/Q308124'), ('castMemberName', 'Anthony LaPaglia')]
[('castMember', 'http://www.wikidata.org/entity/Q1721796'), ('castMemberName', 'Kaitlin Doubleday')]
[('castMember', 'http://www.wikidata.org/entity/Q355163'), ('castMemberName', 'Bruce Davison')]
[('castMember', 'http://www.wikidata.org/entity/Q22704603'), ('castMemberName', 'Lily Knight')]
[('castMember', 'http://www.wikidata.org/entity/Q445325'), ('castMemberName', 'Megan Gallagher')]
[('castMember', 

In [65]:
# Get the super class of TV series 
queryString = """
SELECT DISTINCT ?superClass ?name
WHERE {
   wd:Q5398426 wdt:P279 ?superClass .
   ?superClass sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('superClass', 'http://www.wikidata.org/entity/Q15416'), ('name', 'television program')]
[('superClass', 'http://www.wikidata.org/entity/Q7725310'), ('name', 'series of creative works')]
2


In [66]:
# Get the super class of 'television program' 
queryString = """
SELECT DISTINCT ?superClass ?name
WHERE {
   wd:Q15416 wdt:P279 ?superClass .
   ?superClass sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('superClass', 'http://www.wikidata.org/entity/Q11578774'), ('name', 'broadcasting program')]
[('superClass', 'http://www.wikidata.org/entity/Q2431196'), ('name', 'audiovisual work')]
[('superClass', 'http://www.wikidata.org/entity/Q10301427'), ('name', 'moving image')]
3


In [67]:
# Get all the instances of 'audiovisual work' 
queryString = """
SELECT DISTINCT ?inst ?name
WHERE {
   wd:Q2431196 wdt:P279 ?inst .
   ?inst sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q17537576'), ('name', 'creative work')]
[('inst', 'http://www.wikidata.org/entity/Q194980'), ('name', 'intangible good')]
[('inst', 'http://www.wikidata.org/entity/Q106213450'), ('name', 'time-based work')]
3


In [68]:
# Get info about 'Anthony LaPaglia'
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q308124 ?p ?o .
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('name', 'NUKAT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1220'), ('name', 'Internet Broadway Database person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1266'), ('name', 'AlloCiné person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('name', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
[('p', 'http://www.wikidata.org/prop/direct/P1695'), ('name', 'NLP ID (unique)')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P19'),

In [69]:
# Get info about occuptation of 'Anthony LaPaglia'
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q308124 wdt:P106 ?o .
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q10798782'), ('name', 'television actor')]
[('o', 'http://www.wikidata.org/entity/Q10800557'), ('name', 'film actor')]
[('o', 'http://www.wikidata.org/entity/Q2259451'), ('name', 'stage actor')]
[('o', 'http://www.wikidata.org/entity/Q3282637'), ('name', 'film producer')]
[('o', 'http://www.wikidata.org/entity/Q33999'), ('name', 'actor')]
[('o', 'http://www.wikidata.org/entity/Q937857'), ('name', 'association football player')]
6


In [70]:
# Get subclasses of 'moving work'
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q10301427 wdt:P279 ?sub .
   ?sub sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('name', 'series')]
[('name', 'creative work')]
[('name', 'image')]
3


In [71]:
# Get subclasses of 'moving image'
queryString = """
SELECT DISTINCT ?sub ?name
WHERE {
   wd:Q10301427 wdt:P279 ?sub .
   ?sub sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('sub', 'http://www.wikidata.org/entity/Q20937557'), ('name', 'series')]
[('sub', 'http://www.wikidata.org/entity/Q17537576'), ('name', 'creative work')]
[('sub', 'http://www.wikidata.org/entity/Q478798'), ('name', 'image')]
3


In [72]:
# Get subclasses of 'audiovisual work'
queryString = """
SELECT DISTINCT ?sub ?name
WHERE {
   wd:Q2431196 wdt:P279 ?sub .
   ?sub sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('sub', 'http://www.wikidata.org/entity/Q17537576'), ('name', 'creative work')]
[('sub', 'http://www.wikidata.org/entity/Q194980'), ('name', 'intangible good')]
[('sub', 'http://www.wikidata.org/entity/Q106213450'), ('name', 'time-based work')]
3


In [236]:
# Get all the properties of a 'film actor'
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q10800557 ?p ?o .
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1256'), ('name', 'Iconclass notation')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('name', 'Wikidata property')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P508'), ('name', 'BNCF Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P8814'), ('name',

In [75]:
# Get the subject where derivative work is 'film actor'
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P4969 wd:Q10800557 .
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [76]:
# Get info about properties in which 'Anthony LaPaglia' is involved
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   ?s ?p wd:Q308124 .
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P162'), ('name', 'producer')]
[('p', 'http://www.wikidata.org/prop/direct/P175'), ('name', 'performer')]
[('p', 'http://www.wikidata.org/prop/direct/P26'), ('name', 'spouse')]
[('p', 'http://www.wikidata.org/prop/direct/P3373'), ('name', 'sibling')]
[('p', 'http://www.wikidata.org/prop/direct/P725'), ('name', 'voice actor')]
7


In [78]:
# Get info about performances of 'Anthony LaPaglia'
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P175 wd:Q308124.
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q6113841'), ('name', 'Jack Malone')]
1


In [79]:
# Get info about performances of 'Anthony LaPaglia' as 'Jack Malone'
queryString = """
SELECT DISTINCT ?iof ?name
WHERE {
   ?s wdt:P175 wd:Q308124.
   ?s wdt:P31 ?iof.
   ?iof sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('iof', 'http://www.wikidata.org/entity/Q15773317'), ('name', 'television character')]
1


In [80]:
# Get info about properties in which 'Poppy Mongtgomery' is involved
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   ?s ?p wd:Q235075 .
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P175'), ('name', 'performer')]
[('p', 'http://www.wikidata.org/prop/direct/P451'), ('name', 'unmarried partner')]
3


In [81]:
# Get info about performances of 'Poppy Mongtgomery'
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P175 wd:Q235075.
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q21694294'), ('name', 'Carrie Wells')]
[('s', 'http://www.wikidata.org/entity/Q7408735'), ('name', 'Samantha Spade')]
2


In [83]:
# Get info about instaces of 'audiovisual work'
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P31 wd:Q2431196.
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q622282'), ('name', 'The City on the Edge of Forever')]
[('s', 'http://www.wikidata.org/entity/Q177153'), ('name', 'Die Alpensaga')]
[('s', 'http://www.wikidata.org/entity/Q216655'), ('name', 'Metal Gear')]
[('s', 'http://www.wikidata.org/entity/Q11578774'), ('name', 'broadcasting program')]
[('s', 'http://www.wikidata.org/entity/Q18342381'), ('name', 'LOOP − 60Hz: Transmissions from The Drone Orchestra')]
[('s', 'http://www.wikidata.org/entity/Q240894'), ('name', 'Jakob the Liar')]
[('s', 'http://www.wikidata.org/entity/Q42592465'), ('name', 'The Menagerie, Part 1')]
[('s', 'http://www.wikidata.org/entity/Q104920777'), ('name', "Extreme Weight Loss: Love Can't Weight: Tiffany and Cain")]
[('s', 'http://www.wikidata.org/entity/Q104922925'), ('name', 'American Race: Muslim is the New Black')]
[('s', 'http://www.wikidata.org/entity/Q107751247'), ('name', 'Contemporary Wayang Archive')]
[('s', 'http://www.wikidata.org/entity/Q101062315'), ('n

In [84]:
# Get info about "Looney Tunes Super Stars"
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q6675705 wdt:P31 ?o.
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q17517379'), ('name', 'animated short film')]
[('o', 'http://www.wikidata.org/entity/Q105420'), ('name', 'anthology')]
[('o', 'http://www.wikidata.org/entity/Q2431196'), ('name', 'audiovisual work')]
3


In [85]:
# Get info about super class of "animated short film"
queryString = """
SELECT DISTINCT ?sc ?name
WHERE {
   ?sc wdt:P31 wd:Q17517379.
   ?sc sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('sc', 'http://www.wikidata.org/entity/Q15850464'), ('name', 'The Creation')]
[('sc', 'http://www.wikidata.org/entity/Q16204040'), ('name', 'Minding the Baby')]
[('sc', 'http://www.wikidata.org/entity/Q17040787'), ('name', 'Vida y milagros de Don Fausto')]
[('sc', 'http://www.wikidata.org/entity/Q17092250'), ('name', 'The Idea')]
[('sc', 'http://www.wikidata.org/entity/Q17194141'), ('name', "Uncle Tom's Cabaña")]
[('sc', 'http://www.wikidata.org/entity/Q17311553'), ('name', 'İrfan in University')]
[('sc', 'http://www.wikidata.org/entity/Q17317928'), ('name', 'Amentü Gemisi Nasıl Yürüdü')]
[('sc', 'http://www.wikidata.org/entity/Q17322297'), ('name', 'Tragic Story with Happy Ending')]
[('sc', 'http://www.wikidata.org/entity/Q17634686'), ('name', 'Man on the Chair')]
[('sc', 'http://www.wikidata.org/entity/Q17479054'), ('name', 'Minnie the Moocher')]
[('sc', 'http://www.wikidata.org/entity/Q18155063'), ('name', 'The Sour Puss')]
[('sc', 'http://www.wikidata.org/entity/Q1744972')

In [237]:
# Get info about properties of "animated short film"
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q17517379 ?p ?o.
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('name', 'has list')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P461'), ('name', 'opposite of')]
[('p', 'http://www.wikidata.org/prop/direct/P8408'), ('name', 'KBpedia ID')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]
5


In [88]:
# Get info about properties of "animated short film"
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q17517379 ?p ?o.
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('name', 'has list')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P461'), ('name', 'opposite of')]
[('p', 'http://www.wikidata.org/prop/direct/P8408'), ('name', 'KBpedia ID')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]
5


In [90]:
# Get the super class of TV series 
queryString = """
SELECT DISTINCT ?superClass ?name
WHERE {
   wd:Q5398426 wdt:P279 ?superClass .
   ?superClass sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('superClass', 'http://www.wikidata.org/entity/Q15416'), ('name', 'television program')]
[('superClass', 'http://www.wikidata.org/entity/Q7725310'), ('name', 'series of creative works')]
2


In [91]:
# Get the super class of 'series of creative works' 
queryString = """
SELECT DISTINCT ?superClass ?name
WHERE {
   wd:Q7725310 wdt:P279 ?superClass .
   ?superClass sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('superClass', 'http://www.wikidata.org/entity/Q20937557'), ('name', 'series')]
[('superClass', 'http://www.wikidata.org/entity/Q17489659'), ('name', 'group of works')]
2


In [92]:
# Get the subclass of 'series' 
queryString = """
SELECT DISTINCT ?subClass ?name
WHERE {
   ?subClass wdt:P279 wd:Q20937557 .
   ?subClass sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('subClass', 'http://www.wikidata.org/entity/Q332225'), ('name', 'endgame')]
[('subClass', 'http://www.wikidata.org/entity/Q98391050'), ('name', 'nuclear tests series')]
[('subClass', 'http://www.wikidata.org/entity/Q31836768'), ('name', 'model series')]
[('subClass', 'http://www.wikidata.org/entity/Q133250'), ('name', 'sequence')]
[('subClass', 'http://www.wikidata.org/entity/Q3249551'), ('name', 'process')]
[('subClass', 'http://www.wikidata.org/entity/Q11471'), ('name', 'time')]
[('subClass', 'http://www.wikidata.org/entity/Q52905625'), ('name', 'numismatic serie')]
[('subClass', 'http://www.wikidata.org/entity/Q12538685'), ('name', 'roadshow')]
[('subClass', 'http://www.wikidata.org/entity/Q11424'), ('name', 'film')]
[('subClass', 'http://www.wikidata.org/entity/Q15142889'), ('name', 'weapon family')]
[('subClass', 'http://www.wikidata.org/entity/Q1914636'), ('name', 'activity')]
[('subClass', 'http://www.wikidata.org/entity/Q1326471'), ('name', 'chord progression')]
[('su

In [238]:
# Get an instance of Film 
queryString = """
SELECT DISTINCT ?film ?name
WHERE {
   ?film wdt:P31 wd:Q11424 .
   ?film sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q8178307'), ('name', '15 Park Avenue')]
[('film', 'http://www.wikidata.org/entity/Q8197668'), ('name', 'Idiot Love')]
[('film', 'http://www.wikidata.org/entity/Q319263'), ('name', 'Doctor in the House')]
[('film', 'http://www.wikidata.org/entity/Q8452990'), ('name', "Sunday League - Pepik Hnatek's Final Match")]
[('film', 'http://www.wikidata.org/entity/Q284184'), ('name', 'Ra.One')]
[('film', 'http://www.wikidata.org/entity/Q13637926'), ('name', 'Indra')]
[('film', 'http://www.wikidata.org/entity/Q13638153'), ('name', 'Inner Senses')]
[('film', 'http://www.wikidata.org/entity/Q13645807'), ('name', 'Madhumasam')]
[('film', 'http://www.wikidata.org/entity/Q13647258'), ('name', 'Marupakkam')]
[('film', 'http://www.wikidata.org/entity/Q13677566'), ('name', 'Timberjack')]
10


In [95]:
# Get properties of a film
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   ?film wdt:P31 wd:Q11424 .
   ?film ?p ?o.
   ?p sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2883'), ('name', 'HKMDB film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P7777'), ('name', 'AdoroCinema film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1809'), ('name', 'choreographer')]
[('p', 'http://www.wikidata.org/prop/direct/P7519'), ('name', 'Filmový přehled film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P4606'), ('name', 'National Film Board of Canada film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1040'), ('name', 'film editor')]
[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('name', 'number of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P1237'), ('name', 'Box Office Mojo film ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1265'), ('name', 'AlloCiné film ID')]
10


In [96]:
# Get all properties of a film
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   ?film wdt:P31 wd:Q11424 .
   ?film ?p ?o.
   ?p sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2883'), ('name', 'HKMDB film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P7777'), ('name', 'AdoroCinema film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1809'), ('name', 'choreographer')]
[('p', 'http://www.wikidata.org/prop/direct/P7519'), ('name', 'Filmový přehled film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P4606'), ('name', 'National Film Board of Canada film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1040'), ('name', 'film editor')]
[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('name', 'number of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P1237'), ('name', 'Box Office Mojo film ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1265'), ('name', 'AlloCiné film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/

In [97]:
# Get an instance of Film 
queryString = """
SELECT DISTINCT ?film ?name
WHERE {
   ?film wdt:P31 wd:Q11424 .
   ?film sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q8178307'), ('name', '15 Park Avenue')]
[('film', 'http://www.wikidata.org/entity/Q8197668'), ('name', 'Idiot Love')]
[('film', 'http://www.wikidata.org/entity/Q319263'), ('name', 'Doctor in the House')]
[('film', 'http://www.wikidata.org/entity/Q8452990'), ('name', "Sunday League - Pepik Hnatek's Final Match")]
[('film', 'http://www.wikidata.org/entity/Q284184'), ('name', 'Ra.One')]
[('film', 'http://www.wikidata.org/entity/Q13637926'), ('name', 'Indra')]
[('film', 'http://www.wikidata.org/entity/Q13638153'), ('name', 'Inner Senses')]
[('film', 'http://www.wikidata.org/entity/Q13645807'), ('name', 'Madhumasam')]
[('film', 'http://www.wikidata.org/entity/Q13647258'), ('name', 'Marupakkam')]
[('film', 'http://www.wikidata.org/entity/Q13677566'), ('name', 'Timberjack')]
10


In [98]:
# Get the cast members of the film 'Doctor in the House'
queryString = """
SELECT DISTINCT ?castMember ?name
WHERE {
   wd:Q319263 wdt:P161 ?castMember.
   ?castMember sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q449036'), ('name', 'Joan Sims')]
[('castMember', 'http://www.wikidata.org/entity/Q200840'), ('name', 'Noel Purcell')]
[('castMember', 'http://www.wikidata.org/entity/Q456476'), ('name', 'Suzanne Cloutier')]
[('castMember', 'http://www.wikidata.org/entity/Q432316'), ('name', 'Donald Sinden')]
[('castMember', 'http://www.wikidata.org/entity/Q299190'), ('name', 'Dirk Bogarde')]
[('castMember', 'http://www.wikidata.org/entity/Q321943'), ('name', 'Richard Wattis')]
[('castMember', 'http://www.wikidata.org/entity/Q2844579'), ('name', 'Amy Veness')]
[('castMember', 'http://www.wikidata.org/entity/Q254265'), ('name', 'James Robertson Justice')]
[('castMember', 'http://www.wikidata.org/entity/Q633222'), ('name', 'Kenneth More')]
[('castMember', 'http://www.wikidata.org/entity/Q630728'), ('name', 'Geoffrey Keen')]
10


In [239]:
# Get the films in which 'Joan Sims' played a role
queryString = """
SELECT DISTINCT ?film ?name
WHERE {
   ?film wdt:P161 wd:Q449036.
   ?film sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q319263'), ('name', 'Doctor in the House')]
[('film', 'http://www.wikidata.org/entity/Q1538005'), ('name', "Don't Lose Your Head")]
[('film', 'http://www.wikidata.org/entity/Q1674807'), ('name', 'Follow That Camel')]
[('film', 'http://www.wikidata.org/entity/Q179508'), ('name', 'Carry On Doctor')]
[('film', 'http://www.wikidata.org/entity/Q1199683'), ('name', 'Carry On Behind')]
[('film', 'http://www.wikidata.org/entity/Q1766656'), ('name', 'Carry On Emmannuelle')]
[('film', 'http://www.wikidata.org/entity/Q2146384'), ('name', 'Carry On England')]
[('film', 'http://www.wikidata.org/entity/Q948916'), ('name', 'Carry On')]
[('film', 'http://www.wikidata.org/entity/Q1549606'), ('name', 'One of Our Dinosaurs Is Missing')]
[('film', 'http://www.wikidata.org/entity/Q5287489'), ('name', 'Doctor in Clover')]
10


In [101]:
# Get the films in which 'Joan Sims' played a role
queryString = """
SELECT DISTINCT ?film ?name
WHERE {
   ?film wdt:P161 wd:Q449036.
   ?film sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q319263'), ('name', 'Doctor in the House')]
[('film', 'http://www.wikidata.org/entity/Q1538005'), ('name', "Don't Lose Your Head")]
[('film', 'http://www.wikidata.org/entity/Q1674807'), ('name', 'Follow That Camel')]
[('film', 'http://www.wikidata.org/entity/Q179508'), ('name', 'Carry On Doctor')]
[('film', 'http://www.wikidata.org/entity/Q1199683'), ('name', 'Carry On Behind')]
[('film', 'http://www.wikidata.org/entity/Q1766656'), ('name', 'Carry On Emmannuelle')]
[('film', 'http://www.wikidata.org/entity/Q2146384'), ('name', 'Carry On England')]
[('film', 'http://www.wikidata.org/entity/Q948916'), ('name', 'Carry On')]
[('film', 'http://www.wikidata.org/entity/Q1549606'), ('name', 'One of Our Dinosaurs Is Missing')]
[('film', 'http://www.wikidata.org/entity/Q5287489'), ('name', 'Doctor in Clover')]
[('film', 'http://www.wikidata.org/entity/Q5287492'), ('name', 'Doctor in Love')]
[('film', 'http://www.wikidata.org/entity/Q7762773'), ('

In [103]:
# Get the number of films in which 'Joan Sims' played a role
queryString = """
SELECT (COUNT(DISTINCT ?film) as ?films)
WHERE {
   ?film wdt:P161 wd:Q449036.
   ?film sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('films', '50')]
1


In [104]:
# Get cast about all the episodes in all the seasons of 'Without a Trace'
queryString = """
SELECT ?castMember ?castMemberName
WHERE {
   ?season wdt:P179 wd:Q826477 .        #BGP to retrieve all the seasons
   ?season wdt:P527 ?episode .          #BGP to retrieve all the episodes of a season
   ?episode wdt:P161 ?castMember.       #BGP to retireve all cast members that participated in the episode
   ?castMember sc:name ?castMemberName.
}
GROUP BY ?castMember ?castMemberName
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q308124'), ('castMemberName', 'Anthony LaPaglia')]
[('castMember', 'http://www.wikidata.org/entity/Q459638'), ('castMemberName', 'Stacy Edwards')]
[('castMember', 'http://www.wikidata.org/entity/Q706513'), ('castMemberName', 'Charles S. Dutton')]
[('castMember', 'http://www.wikidata.org/entity/Q5363239'), ('castMemberName', 'Elizabeth Morehead')]
[('castMember', 'http://www.wikidata.org/entity/Q64684485'), ('castMemberName', 'Caris Vujcec')]
[('castMember', 'http://www.wikidata.org/entity/Q1721796'), ('castMemberName', 'Kaitlin Doubleday')]
[('castMember', 'http://www.wikidata.org/entity/Q20651783'), ('castMemberName', 'Jenica Bergere')]
[('castMember', 'http://www.wikidata.org/entity/Q235075'), ('castMemberName', 'Poppy Montgomery')]
[('castMember', 'http://www.wikidata.org/entity/Q297128'), ('castMemberName', 'David Henrie')]
[('castMember', 'http://www.wikidata.org/entity/Q503040'), ('castMemberName', 'Eric Close')]
[('castMemb

In [107]:
# Get the number of casts in which 'Anthony LaPaglia' played a role
queryString = """
SELECT (COUNT(DISTINCT ?film) as ?films)
WHERE {
   ?film wdt:P161 wd:Q308124.
   ?film sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('films', '216')]
1


In [109]:
# Get the casts in which 'Anthony LaPaglia' played a role
queryString = """
SELECT ?partIn ?name
WHERE {
   ?partIn wdt:P161 wd:Q308124.
   ?partIn sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('partIn', 'http://www.wikidata.org/entity/Q255376'), ('name', 'The Client')]
[('partIn', 'http://www.wikidata.org/entity/Q21511509'), ('name', 'Annabelle: Creation')]
[('partIn', 'http://www.wikidata.org/entity/Q12126147'), ('name', 'Mental')]
[('partIn', 'http://www.wikidata.org/entity/Q3805682'), ('name', 'Jack the Dog')]
[('partIn', 'http://www.wikidata.org/entity/Q826477'), ('name', 'Without a Trace')]
[('partIn', 'http://www.wikidata.org/entity/Q1352085'), ('name', 'Analyze That')]
[('partIn', 'http://www.wikidata.org/entity/Q7728500'), ('name', 'The Custodian')]
[('partIn', 'http://www.wikidata.org/entity/Q16954197'), ('name', 'The Code')]
[('partIn', 'http://www.wikidata.org/entity/Q3549263'), ('name', 'Underground: The Julian Assange Story')]
[('partIn', 'http://www.wikidata.org/entity/Q1570302'), ('name', 'Lansky')]
[('partIn', 'http://www.wikidata.org/entity/Q1125349'), ('name', 'Autumn in New York')]
[('partIn', 'http://www.wikidata.org/entity/Q52667904'), ('name',

In [110]:
# Get the casts in which 'Anthony LaPaglia' played a role
queryString = """
SELECT ?partIn ?name ?superclassName
WHERE {
   ?partIn wdt:P161 wd:Q308124.
   ?partIn wdt:P31 ?superclass.
   ?superclass sc:name ?superclassName. 
   ?partIn sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('partIn', 'http://www.wikidata.org/entity/Q255376'), ('name', 'The Client'), ('superclassName', 'film')]
[('partIn', 'http://www.wikidata.org/entity/Q21511509'), ('name', 'Annabelle: Creation'), ('superclassName', 'film')]
[('partIn', 'http://www.wikidata.org/entity/Q12126147'), ('name', 'Mental'), ('superclassName', 'film')]
[('partIn', 'http://www.wikidata.org/entity/Q3805682'), ('name', 'Jack the Dog'), ('superclassName', 'film')]
[('partIn', 'http://www.wikidata.org/entity/Q826477'), ('name', 'Without a Trace'), ('superclassName', 'television series')]
[('partIn', 'http://www.wikidata.org/entity/Q1352085'), ('name', 'Analyze That'), ('superclassName', 'film')]
[('partIn', 'http://www.wikidata.org/entity/Q7728500'), ('name', 'The Custodian'), ('superclassName', 'film')]
[('partIn', 'http://www.wikidata.org/entity/Q16954197'), ('name', 'The Code'), ('superclassName', 'television series')]
[('partIn', 'http://www.wikidata.org/entity/Q3549263'), ('name', 'Underground: The Jul

In [111]:
# Get the casts in which 'Anthony LaPaglia' played a role
queryString = """
SELECT ?partIn ?name ?superclass ?superclassName
WHERE {
   ?partIn wdt:P161 wd:Q308124.
   ?partIn wdt:P31 ?superclass.
   ?superclass sc:name ?superclassName. 
   ?partIn sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('partIn', 'http://www.wikidata.org/entity/Q255376'), ('name', 'The Client'), ('superclass', 'http://www.wikidata.org/entity/Q11424'), ('superclassName', 'film')]
[('partIn', 'http://www.wikidata.org/entity/Q21511509'), ('name', 'Annabelle: Creation'), ('superclass', 'http://www.wikidata.org/entity/Q11424'), ('superclassName', 'film')]
[('partIn', 'http://www.wikidata.org/entity/Q12126147'), ('name', 'Mental'), ('superclass', 'http://www.wikidata.org/entity/Q11424'), ('superclassName', 'film')]
[('partIn', 'http://www.wikidata.org/entity/Q3805682'), ('name', 'Jack the Dog'), ('superclass', 'http://www.wikidata.org/entity/Q11424'), ('superclassName', 'film')]
[('partIn', 'http://www.wikidata.org/entity/Q826477'), ('name', 'Without a Trace'), ('superclass', 'http://www.wikidata.org/entity/Q5398426'), ('superclassName', 'television series')]
[('partIn', 'http://www.wikidata.org/entity/Q1352085'), ('name', 'Analyze That'), ('superclass', 'http://www.wikidata.org/entity/Q11424'), (

In [112]:
# Get the film casts in which 'Anthony LaPaglia' played a role
queryString = """
SELECT ?partIn ?name
WHERE {
   ?partIn wdt:P161 wd:Q308124.
   ?partIn wdt:P31 ?sc.
   ?partIn sc:name ?name.
   FILTER (?sc = wd:Q11424)
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('partIn', 'http://www.wikidata.org/entity/Q255376'), ('name', 'The Client')]
[('partIn', 'http://www.wikidata.org/entity/Q21511509'), ('name', 'Annabelle: Creation')]
[('partIn', 'http://www.wikidata.org/entity/Q12126147'), ('name', 'Mental')]
[('partIn', 'http://www.wikidata.org/entity/Q3805682'), ('name', 'Jack the Dog')]
[('partIn', 'http://www.wikidata.org/entity/Q1352085'), ('name', 'Analyze That')]
[('partIn', 'http://www.wikidata.org/entity/Q7728500'), ('name', 'The Custodian')]
[('partIn', 'http://www.wikidata.org/entity/Q1125349'), ('name', 'Autumn in New York')]
[('partIn', 'http://www.wikidata.org/entity/Q1167468'), ('name', 'Chameleon')]
[('partIn', 'http://www.wikidata.org/entity/Q2056364'), ('name', 'Company Man')]
[('partIn', 'http://www.wikidata.org/entity/Q786585'), ('name', 'Sweet and Lowdown')]
10


In [114]:
# Get the number of film casts in which 'Anthony LaPaglia' played a role
queryString = """
SELECT (COUNT(DISTINCT ?partIn) as ?films)
WHERE {
   ?partIn wdt:P161 wd:Q308124.
   ?partIn wdt:P31 ?sc.
   FILTER (?sc = wd:Q11424)
   ?partIn sc:name ?name.

}
"""

print("Results")
x=run_query(queryString)

Results
[('films', '47')]
1


In [119]:
# Get the number of film casts in which 'Stacy Edwards' played a role
queryString = """
SELECT (COUNT(DISTINCT ?partIn) as ?films)
WHERE {
   ?partIn wdt:P161 wd:Q459638.
   ?partIn wdt:P31 ?sc.
   FILTER (?sc = wd:Q11424)
   ?partIn sc:name ?name.

}
"""

print("Results")
x=run_query(queryString)

Results
[('films', '12')]
1


In [129]:
# Get the number of films of all the cast members of 'Without a Trace'
queryString = """
SELECT ?castMember ?castMemberName (COUNT(DISTINCT ?partIn) as ?films)
WHERE {
   ?season wdt:P179 wd:Q826477 .        #BGP to retrieve all the seasons
   ?season wdt:P527 ?episode .          #BGP to retrieve all the episodes of a season
   ?episode wdt:P161 ?castMember.       #BGP to retireve all cast members that participated in the episode
   ?castMember sc:name ?castMemberName.

   ?partIn wdt:P161 ?castMember.        #BGP to retrieve all the parts of a cast member 
   ?partIn wdt:P31 ?sc.                 #BGP used to filter only films
   FILTER (?sc = wd:Q11424)
   
}
GROUP BY ?castMember ?castMemberName
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q18618690'), ('castMemberName', 'John Livingston'), ('films', '5')]
[('castMember', 'http://www.wikidata.org/entity/Q355163'), ('castMemberName', 'Bruce Davison'), ('films', '56')]
[('castMember', 'http://www.wikidata.org/entity/Q297128'), ('castMemberName', 'David Henrie'), ('films', '5')]
[('castMember', 'http://www.wikidata.org/entity/Q459638'), ('castMemberName', 'Stacy Edwards'), ('films', '13')]
[('castMember', 'http://www.wikidata.org/entity/Q1125651'), ('castMemberName', 'Thom Barry'), ('films', '15')]
[('castMember', 'http://www.wikidata.org/entity/Q4769249'), ('castMemberName', 'Annie Burgstede'), ('films', '2')]
[('castMember', 'http://www.wikidata.org/entity/Q1721796'), ('castMemberName', 'Kaitlin Doubleday'), ('films', '3')]
[('castMember', 'http://www.wikidata.org/entity/Q2927620'), ('castMemberName', 'Denise Dowse'), ('films', '1')]
[('castMember', 'http://www.wikidata.org/entity/Q430872'), ('castMemberName', 'Maria

Final query for this task

In [248]:
queryString = """
SELECT ?castMember ?castMemberName (COUNT(DISTINCT ?partIn) as ?films)
WHERE {
   ?season wdt:P179 wd:Q826477 .        #BGP to retrieve all the seasons
   ?season wdt:P527 ?episode .          #BGP to retrieve all the episodes of a season
   ?episode wdt:P161 ?castMember.       #BGP to retireve all cast members that participated in the episode
   ?castMember sc:name ?castMemberName.

   ?partIn wdt:P161 ?castMember.        #BGP to retrieve all the parts of a cast member 
   ?partIn wdt:P31 ?sc.                 #BGP used to filter only films
   FILTER (?sc = wd:Q11424)
   
}
GROUP BY ?castMember ?castMemberName
ORDER BY DESC(?films)
LIMIT 1
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q355163'), ('castMemberName', 'Bruce Davison'), ('films', '56')]
1


## Task 5
Compare Without a Trace with the tv series "Cold Case" in terms of number of seasons, episods and cast members (the result set must be two elements -one for each tv series- of tv series IRI, label, #seasons, #episodes and #cast members).

In [130]:
# Get all the properties of 'Without a Trace'
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q826477 ?p ?obj .
   ?p sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('name', 'number of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1267'), ('name', 'AlloCiné series ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('name', 'title')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P1562'), ('name', 'AllMovie title ID')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P170'), ('name', 'creator')]
[('p', 'http://www.wikidata.org/prop/direct/P1712'), ('name', 'Metacritic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1811'), ('name', 'list of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P2047'), ('name', 'duration')]
[('p', 'http://www.wikidata.org/prop/direct/P2

In [136]:
# Get cast about all the episodes in all the seasons of 'Without a Trace'
queryString = """
SELECT (COUNT(DISTINCT(?castMember)) as ?castMembers)
WHERE {
   ?season wdt:P179 wd:Q826477 .        #BGP to retrieve all the seasons
   ?season wdt:P527 ?episode .          #BGP to retrieve all the episodes of a season
   ?episode wdt:P161 ?castMember.       #BGP to retireve all cast members that participated in the episode
}
"""

print("Results")
x=run_query(queryString)

Results
[('castMembers', '30')]
1


In [137]:
# Get all the properties of 'Without a Trace'
queryString = """
SELECT DISTINCT ?episodes ?seasons
WHERE {
   wd:Q826477 wdt:P1113 ?episodes;
              wdt:P2437 ?seasons .
}
"""

print("Results")
x=run_query(queryString)

Results
[('episodes', '160'), ('seasons', '7')]
1


In [140]:
# Get the class of 'Without a Trace'
queryString = """
SELECT DISTINCT ?class ?name
WHERE {
   wd:Q826477 wdt:P31 ?class.
   ?class sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q5398426'), ('name', 'television series')]
1


In [142]:
# Get tv series
queryString = """
SELECT DISTINCT ?tvSerie ?name
WHERE {
   ?tvSerie wdt:P31 wd:Q5398426.
   ?tvSerie sc:name ?name.
}
LIMIT 5
"""

print("Results")
x=run_query(queryString)

Results
[('tvSerie', 'http://www.wikidata.org/entity/Q1154217'), ('name', 'Ironfist Chinmi')]
[('tvSerie', 'http://www.wikidata.org/entity/Q691528'), ('name', 'Wallace and Gromit')]
[('tvSerie', 'http://www.wikidata.org/entity/Q8193809'), ('name', 'Shomuni')]
[('tvSerie', 'http://www.wikidata.org/entity/Q33323'), ('name', 'Everybody Hates Chris')]
[('tvSerie', 'http://www.wikidata.org/entity/Q500769'), ('name', 'Freaks and Geeks')]
5


In [144]:
# Get info about 'Without a Trace'
queryString = """
SELECT ?tvSerie ?name ?seasons ?episodes
WHERE {
   ?tvSerie wdt:P31 wd:Q5398426.
   ?tvSerie sc:name ?name.
   
   FILTER(?tvSerie = wd:Q826477) #Filter TV serie 'Without a Trace'
   ?tvSerie wdt:P2437 ?seasons.
   ?tvSerie wdt:P1113 ?episodes.
}
"""

print("Results")
x=run_query(queryString)

Results
[('tvSerie', 'http://www.wikidata.org/entity/Q826477'), ('name', 'Without a Trace'), ('seasons', '7'), ('episodes', '160')]
1


In [148]:
# Get info about 'Without a Trace' or 'Cold Case'
queryString = """
SELECT ?tvSerie ?name ?seasons ?episodes
WHERE {
   ?tvSerie wdt:P31 wd:Q5398426.
   ?tvSerie sc:name ?name.
   
   FILTER(?tvSerie = wd:Q826477|| ?tvSerie = wd:Q733960)
   ?tvSerie wdt:P2437 ?seasons.
   ?tvSerie wdt:P1113 ?episodes.
   
}
"""

print("Results")
x=run_query(queryString)

Results
[('tvSerie', 'http://www.wikidata.org/entity/Q733960'), ('name', 'Cold Case'), ('seasons', '7'), ('episodes', '156')]
[('tvSerie', 'http://www.wikidata.org/entity/Q826477'), ('name', 'Without a Trace'), ('seasons', '7'), ('episodes', '160')]
2


Final query for this task

In [246]:
# Get info about 'Without a Trace' or 'Cold Case'
queryString = """
SELECT ?tvSerie ?name ?seasons ?episodes (COUNT(DISTINCT(?castMember)) as ?castMembers)
WHERE {
   ?tvSerie wdt:P31 wd:Q5398426.
   ?tvSerie sc:name ?name.
   
   FILTER(?tvSerie = wd:Q826477|| ?tvSerie = wd:Q733960)
   ?tvSerie wdt:P2437 ?seasons.
   ?tvSerie wdt:P1113 ?episodes.
   
   ?season wdt:P179 ?tvSerie .          #BGP to retrieve all the seasons
   ?season wdt:P527 ?episode .          #BGP to retrieve all the episodes of a season
   ?episode wdt:P161 ?castMember.       #BGP to retireve all cast members that participated in the episode
   
}
"""

print("Results")
x=run_query(queryString)

Results
[('tvSerie', 'http://www.wikidata.org/entity/Q826477'), ('name', 'Without a Trace'), ('seasons', '7'), ('episodes', '160'), ('castMembers', '30')]
[('tvSerie', 'http://www.wikidata.org/entity/Q733960'), ('name', 'Cold Case'), ('seasons', '7'), ('episodes', '156'), ('castMembers', '674')]
2


## Task 6
Return the actors who are members of the cast of Without a Trace have [Kavin Bacon number](https://en.wikipedia.org/wiki/Six_Degrees_of_Kevin_Bacon#:~:text=Kevin%20Bacon%20himself%20has%20a,Bacon%20number%20is%20N%2B1.) equal to 2 (the result set must be a list of couples actor/actress IRI and label).

In [151]:
# Get info about occuptation of 'Anthony LaPaglia'
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q308124 wdt:P106 ?o .
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q10798782'), ('name', 'television actor')]
[('o', 'http://www.wikidata.org/entity/Q10800557'), ('name', 'film actor')]
[('o', 'http://www.wikidata.org/entity/Q2259451'), ('name', 'stage actor')]
[('o', 'http://www.wikidata.org/entity/Q3282637'), ('name', 'film producer')]
[('o', 'http://www.wikidata.org/entity/Q33999'), ('name', 'actor')]
[('o', 'http://www.wikidata.org/entity/Q937857'), ('name', 'association football player')]
6


In [241]:
# Get some properties of 'actor'
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q33999 ?p ?o .
   ?p sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1256'), ('name', 'Iconclass notation')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('name', 'Wikidata property')]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('name', 'equivalent class')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
10


In [154]:
# Get info about 'actor'
queryString = """
SELECT DISTINCT ?superClass ?name
WHERE {
   wd:Q33999 wdt:P31 ?superClass .
   ?superClass sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('superClass', 'http://www.wikidata.org/entity/Q12737077'), ('name', 'occupation')]
[('superClass', 'http://www.wikidata.org/entity/Q28640'), ('name', 'profession')]
[('superClass', 'http://www.wikidata.org/entity/Q108300140'), ('name', 'occupation group according to ISCO')]
3


In [156]:
# Get info about superclass of 'Anthony LaPaglia'
queryString = """
SELECT DISTINCT ?superClass ?name
WHERE {
   wd:Q308124 wdt:P31 ?superClass .
   ?superClass sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('superClass', 'http://www.wikidata.org/entity/Q5'), ('name', 'human')]
1


In [157]:
# Get properties of 'human'
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q5 ?p ?o .
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P5806'), ('name', 'SNOMED CT identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('name', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1056'), ('name', 'product or material produced')]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1256'), ('name', 'Iconclass notation')]
[('p', 'http://www.wikidata.org/prop/direct/P129'), ('name', 'physically interacts with')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Onlin

In [158]:
# Get some instances of 'human'
queryString = """
SELECT DISTINCT ?human ?name
WHERE {
   ?human wdt:P31 wd:Q5 .
   ?human sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('human', 'http://www.wikidata.org/entity/Q1002728'), ('name', 'Fonsi Nieto')]
[('human', 'http://www.wikidata.org/entity/Q100343342'), ('name', 'Lubomír Stoklásek')]
[('human', 'http://www.wikidata.org/entity/Q102115260'), ('name', 'Kojiro Kobayashi')]
[('human', 'http://www.wikidata.org/entity/Q102118153'), ('name', 'Miguel E. M. Herrera')]
[('human', 'http://www.wikidata.org/entity/Q105753880'), ('name', 'Mateo Egea')]
[('human', 'http://www.wikidata.org/entity/Q107520918'), ('name', 'Jerónimo Latorre')]
[('human', 'http://www.wikidata.org/entity/Q11404285'), ('name', 'Mikiko Kitano')]
[('human', 'http://www.wikidata.org/entity/Q11520274'), ('name', 'Honjō Fusanaga')]
[('human', 'http://www.wikidata.org/entity/Q11562694'), ('name', 'Takayoshi Watanabe')]
[('human', 'http://www.wikidata.org/entity/Q11685026'), ('name', 'Joan de Gènova')]
10


In [170]:
# Get properties of an 'human'
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q1002728 ?p ?o.
   ?p sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3928'), ('name', 'MotoGP racer ID')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('name', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('name', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('name', 'place of birth')]
[('p', 'http://www.wikidata.org/prop/direct/P21'), ('name', 'sex or gender')]
[('p', 'http://www.wikidata.org/prop/direct/P27'), ('name', 'country of citizenship')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P451'), ('name', 'unmarried partner')]
[('p', 'http://www.wikidata.org/prop/direct/P551'), ('name', 'residence')]
[('p', 'http://www.wikidata.

In [180]:
# Get the properties of a 'human' with profession 'actor'
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   ?human wdt:P31 wd:Q5 .      #BGP to identify an human
   ?human wdt:P106 wd:Q33999.  #BGP to identify an human with profession 'actor'
   
   ?human ?p ?o .
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3269'), ('name', 'Fotografen.nl ID')]
[('p', 'http://www.wikidata.org/prop/direct/P6988'), ('name', 'Hungarian National Namespace person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P8577'), ('name', 'ICP artist ID')]
[('p', 'http://www.wikidata.org/prop/direct/P6891'), ('name', 'National Film Board of Canada director identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P3140'), ('name', 'SourehCinema person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P102'), ('name', 'member of political party')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native langua

In [182]:
# Get 'qualities' of a 'human'
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   ?human wdt:P31 wd:Q5 .      #BGP to identify an human
   ?human wdt:P1552 ?o .
   ?o sc:name ?name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q81867'), ('name', 'albinism')]
[('o', 'http://www.wikidata.org/entity/Q2985549'), ('name', 'mononymous person')]
[('o', 'http://www.wikidata.org/entity/Q72941682'), ('name', 'tattoo')]
[('o', 'http://www.wikidata.org/entity/Q1062083'), ('name', 'billionaire')]
[('o', 'http://www.wikidata.org/entity/Q1775415'), ('name', 'feminine')]
[('o', 'http://www.wikidata.org/entity/Q388505'), ('name', 'missing person')]
[('o', 'http://www.wikidata.org/entity/Q55071270'), ('name', 'classical training')]
[('o', 'http://www.wikidata.org/entity/Q205178'), ('name', 'child prodigy')]
[('o', 'http://www.wikidata.org/entity/Q157943'), ('name', 'body piercing')]
[('o', 'http://www.wikidata.org/entity/Q1087232'), ('name', 'Dyslalia')]
[('o', 'http://www.wikidata.org/entity/Q1747829'), ('name', 'notname')]
[('o', 'http://www.wikidata.org/entity/Q213250'), ('name', 'absolute pitch')]
[('o', 'http://www.wikidata.org/entity/Q63441625'), ('name', 'giantness')]
[('o

In [183]:
# Get the properties of 'Anthony LaPaglia'
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q308124 ?p ?o .
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('name', 'NUKAT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1220'), ('name', 'Internet Broadway Database person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1266'), ('name', 'AlloCiné person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('name', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
[('p', 'http://www.wikidata.org/prop/direct/P1695'), ('name', 'NLP ID (unique)')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P19'),

In [186]:
# Get super class of 'human'
queryString = """
SELECT DISTINCT ?sc ?name
WHERE {
   wd:Q5 wdt:P279 ?sc .
   ?sc sc:name ?name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('sc', 'http://www.wikidata.org/entity/Q215627'), ('name', 'person')]
[('sc', 'http://www.wikidata.org/entity/Q154954'), ('name', 'natural person')]
[('sc', 'http://www.wikidata.org/entity/Q164509'), ('name', 'omnivore')]
3


In [187]:
# Get properties of 'person'
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q215627 ?p ?o .
   ?p sc:name ?name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1368'), ('name', 'LNB ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('name', 'has quality')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('name', 'Wikidata property')]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('name', 'equivalent class')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('name', 'Ba

In [188]:
# Get 'qualities' of 'person'
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q215627 wdt:P1552 ?o .
   ?o sc:name ?name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q61002'), ('name', 'pseudonym')]
[('o', 'http://www.wikidata.org/entity/Q202444'), ('name', 'given name')]
[('o', 'http://www.wikidata.org/entity/Q37787614'), ('name', 'person property')]
[('o', 'http://www.wikidata.org/entity/Q641118'), ('name', 'personality')]
[('o', 'http://www.wikidata.org/entity/Q2867027'), ('name', 'personal life')]
[('o', 'http://www.wikidata.org/entity/Q185836'), ('name', 'age of a person')]
[('o', 'http://www.wikidata.org/entity/Q2389905'), ('name', 'date of birth')]
[('o', 'http://www.wikidata.org/entity/Q59157079'), ('name', 'biographic information')]
[('o', 'http://www.wikidata.org/entity/Q3968640'), ('name', 'mental state')]
[('o', 'http://www.wikidata.org/entity/Q101352'), ('name', 'family name')]
[('o', 'http://www.wikidata.org/entity/Q3738275'), ('name', 'adaptive capacity')]
[('o', 'http://www.wikidata.org/entity/Q48277'), ('name', 'gender')]
[('o', 'http://www.wikidata.org/entity/Q2254571'), ('name', 'per

In [198]:
# Get some instances of 'human' which name contains 'Kevin'
queryString = """
SELECT DISTINCT ?human ?name
WHERE {
   ?human wdt:P31 wd:Q5 .
   ?human sc:name ?name.
   FILTER REGEX(?name, "[Kk]evin*"). 
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('human', 'http://www.wikidata.org/entity/Q545844'), ('name', 'Sergei Yutkevich')]
[('human', 'http://www.wikidata.org/entity/Q22695818'), ('name', 'Kevin Alexander')]
[('human', 'http://www.wikidata.org/entity/Q15462524'), ('name', 'Kevin Wildes')]
[('human', 'http://www.wikidata.org/entity/Q603999'), ('name', 'Kevin Poulsen')]
[('human', 'http://www.wikidata.org/entity/Q15907071'), ('name', 'Kevin van Kippersluis')]
[('human', 'http://www.wikidata.org/entity/Q718313'), ('name', 'Kevin Tsai')]
[('human', 'http://www.wikidata.org/entity/Q16017360'), ('name', 'Kevin Corby')]
[('human', 'http://www.wikidata.org/entity/Q102289621'), ('name', 'Kevin Edward Fu')]
[('human', 'http://www.wikidata.org/entity/Q16170868'), ('name', 'Knut Bakkevig')]
[('human', 'http://www.wikidata.org/entity/Q16193414'), ('name', 'Irina Valyukevich')]
10


In [201]:
# Get some instances of 'human' which name contains 'Kevin' and 'Bacon'
queryString = """
SELECT DISTINCT ?human ?name
WHERE {
   ?human wdt:P31 wd:Q5 .
   ?human sc:name ?name.
   FILTER REGEX(?name, "[Kk]evin*"). 
   FILTER REGEX(?name, "[Bb]acon*"). 
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('human', 'http://www.wikidata.org/entity/Q3454165'), ('name', 'Kevin Bacon')]
1


In [202]:
# Get info about occuptation of 'Kevin Bacon'
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q3454165 wdt:P106 ?o .
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q10798782'), ('name', 'television actor')]
[('o', 'http://www.wikidata.org/entity/Q10800557'), ('name', 'film actor')]
[('o', 'http://www.wikidata.org/entity/Q13235160'), ('name', 'manufacturer')]
[('o', 'http://www.wikidata.org/entity/Q2259451'), ('name', 'stage actor')]
[('o', 'http://www.wikidata.org/entity/Q2405480'), ('name', 'voice actor')]
[('o', 'http://www.wikidata.org/entity/Q2526255'), ('name', 'film director')]
[('o', 'http://www.wikidata.org/entity/Q3282637'), ('name', 'film producer')]
[('o', 'http://www.wikidata.org/entity/Q33999'), ('name', 'actor')]
[('o', 'http://www.wikidata.org/entity/Q3455803'), ('name', 'director')]
[('o', 'http://www.wikidata.org/entity/Q639669'), ('name', 'musician')]
[('o', 'http://www.wikidata.org/entity/Q948329'), ('name', 'character actor')]
11


In [242]:
# Get all the films in which 'Kevin Bacon' appears as 'cast member'
queryString = """
SELECT DISTINCT ?appearsIn ?name
WHERE {
    ?appearsIn wdt:P161 wd:Q3454165.
    ?appearsIn sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('appearsIn', 'http://www.wikidata.org/entity/Q122113'), ('name', 'Super')]
[('appearsIn', 'http://www.wikidata.org/entity/Q463927'), ('name', 'Diner')]
[('appearsIn', 'http://www.wikidata.org/entity/Q636062'), ('name', 'Loverboy')]
[('appearsIn', 'http://www.wikidata.org/entity/Q691672'), ('name', 'Frost/Nixon')]
[('appearsIn', 'http://www.wikidata.org/entity/Q1917925'), ('name', 'My Dog Skip')]
[('appearsIn', 'http://www.wikidata.org/entity/Q5265117'), ('name', 'Destination Anywhere (film)')]
[('appearsIn', 'http://www.wikidata.org/entity/Q1154802'), ('name', 'Murder in the First')]
[('appearsIn', 'http://www.wikidata.org/entity/Q1243029'), ('name', 'Friday the 13th')]
[('appearsIn', 'http://www.wikidata.org/entity/Q108946'), ('name', 'A Few Good Men')]
[('appearsIn', 'http://www.wikidata.org/entity/Q223596'), ('name', 'X-Men: First Class')]
10


In [250]:
# Get all the films in which 'Kevin Bacon' appears as 'cast member'
queryString = """
SELECT DISTINCT ?appearsIn ?name
WHERE {
    wd:Q3454165 ^wdt:P161 ?appearsIn.
    ?appearsIn sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('appearsIn', 'http://www.wikidata.org/entity/Q122113'), ('name', 'Super')]
[('appearsIn', 'http://www.wikidata.org/entity/Q463927'), ('name', 'Diner')]
[('appearsIn', 'http://www.wikidata.org/entity/Q636062'), ('name', 'Loverboy')]
[('appearsIn', 'http://www.wikidata.org/entity/Q691672'), ('name', 'Frost/Nixon')]
[('appearsIn', 'http://www.wikidata.org/entity/Q1917925'), ('name', 'My Dog Skip')]
[('appearsIn', 'http://www.wikidata.org/entity/Q5265117'), ('name', 'Destination Anywhere (film)')]
[('appearsIn', 'http://www.wikidata.org/entity/Q1154802'), ('name', 'Murder in the First')]
[('appearsIn', 'http://www.wikidata.org/entity/Q1243029'), ('name', 'Friday the 13th')]
[('appearsIn', 'http://www.wikidata.org/entity/Q108946'), ('name', 'A Few Good Men')]
[('appearsIn', 'http://www.wikidata.org/entity/Q223596'), ('name', 'X-Men: First Class')]
10


In [208]:
# Get the casts of 'Anthony LaPaglia'
queryString = """
SELECT DISTINCT ?cast ?name
WHERE {
   ?cast wdt:P161 wd:Q308124  .
   ?cast sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('cast', 'http://www.wikidata.org/entity/Q255376'), ('name', 'The Client')]
[('cast', 'http://www.wikidata.org/entity/Q21511509'), ('name', 'Annabelle: Creation')]
[('cast', 'http://www.wikidata.org/entity/Q12126147'), ('name', 'Mental')]
[('cast', 'http://www.wikidata.org/entity/Q3805682'), ('name', 'Jack the Dog')]
[('cast', 'http://www.wikidata.org/entity/Q826477'), ('name', 'Without a Trace')]
[('cast', 'http://www.wikidata.org/entity/Q1352085'), ('name', 'Analyze That')]
[('cast', 'http://www.wikidata.org/entity/Q7728500'), ('name', 'The Custodian')]
[('cast', 'http://www.wikidata.org/entity/Q16954197'), ('name', 'The Code')]
[('cast', 'http://www.wikidata.org/entity/Q3549263'), ('name', 'Underground: The Julian Assange Story')]
[('cast', 'http://www.wikidata.org/entity/Q1570302'), ('name', 'Lansky')]
10


In [210]:
# Get the cast members of 'The Client'
queryString = """
SELECT DISTINCT ?castMember ?name
WHERE {
   wd:Q255376 wdt:P161 ?castMember  .
   ?castMember sc:name ?name .
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q133050'), ('name', 'Susan Sarandon')]
[('castMember', 'http://www.wikidata.org/entity/Q573332'), ('name', 'Anthony Heald')]
[('castMember', 'http://www.wikidata.org/entity/Q352180'), ('name', 'Bradley Whitford')]
[('castMember', 'http://www.wikidata.org/entity/Q1258012'), ('name', 'Kim Coates')]
[('castMember', 'http://www.wikidata.org/entity/Q1372770'), ('name', 'William Sanderson')]
[('castMember', 'http://www.wikidata.org/entity/Q224159'), ('name', 'William H. Macy')]
[('castMember', 'http://www.wikidata.org/entity/Q229957'), ('name', 'Mary-Louise Parker')]
[('castMember', 'http://www.wikidata.org/entity/Q308124'), ('name', 'Anthony LaPaglia')]
[('castMember', 'http://www.wikidata.org/entity/Q314924'), ('name', 'Anthony Edwards')]
[('castMember', 'http://www.wikidata.org/entity/Q361400'), ('name', 'Will Patton')]
10


In [211]:
# Check if among all the cast members of 'The Client' there is one which has directly worked with 'Kevin Bacon'
queryString = """
SELECT DISTINCT ?castMember ?name
WHERE {
   wd:Q255376 wdt:P161 ?castMember  .
   ?castMember sc:name ?name .
   
   FILTER(?castMember = ?cm)
   {
      SELECT DISTINCT ?cm
      WHERE {
          ?cast wdt:P161 ?cm .         #BGP to get the casts for an actor
          wd:Q3454165 wdt:161 ?cast .  #BGP to specify if Kevin Bacon is part of the cast
      }
   }
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
Empty


Final query for this task

In [252]:
# Get cast about all the episodes in all the seasons of 'Without a Trace' with 'Kevin Bacon' number equal to 2
queryString = """
SELECT ?castMember ?castMemberName
WHERE {
   ?season wdt:P179 wd:Q826477 .        #BGP to retrieve all the seasons
   ?season wdt:P527 ?episode .          #BGP to retrieve all the episodes of a season
   ?episode wdt:P161 ?castMember.       #BGP to retireve all cast members that participated in the episode
   ?castMember sc:name ?castMemberName.
   
   ?castMember (^wdt:P161 / wdt:P161){2} wd:Q3454165. 
   
}
GROUP BY ?castMember ?castMemberName
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q308124'), ('castMemberName', 'Anthony LaPaglia')]
[('castMember', 'http://www.wikidata.org/entity/Q459638'), ('castMemberName', 'Stacy Edwards')]
[('castMember', 'http://www.wikidata.org/entity/Q706513'), ('castMemberName', 'Charles S. Dutton')]
[('castMember', 'http://www.wikidata.org/entity/Q5363239'), ('castMemberName', 'Elizabeth Morehead')]
[('castMember', 'http://www.wikidata.org/entity/Q64684485'), ('castMemberName', 'Caris Vujcec')]
[('castMember', 'http://www.wikidata.org/entity/Q297128'), ('castMemberName', 'David Henrie')]
[('castMember', 'http://www.wikidata.org/entity/Q235075'), ('castMemberName', 'Poppy Montgomery')]
[('castMember', 'http://www.wikidata.org/entity/Q1721796'), ('castMemberName', 'Kaitlin Doubleday')]
[('castMember', 'http://www.wikidata.org/entity/Q20651783'), ('castMemberName', 'Jenica Bergere')]
[('castMember', 'http://www.wikidata.org/entity/Q503040'), ('castMemberName', 'Eric Close')]
[('castMemb

## Task 7
Consider the actors who are members of the cast of Cold Case. Amongst the tv series which these actors acted return only those which received more than 2 awards (the result set must be triples of tv series IRI, label, #awards won).

In [215]:
# Get cast about all the episodes in all the seasons of 'Cold Case'
queryString = """
SELECT ?castMember ?castMemberName
WHERE {
   ?season wdt:P179 wd:Q733960 .        #BGP to retrieve all the seasons of Cold Case
   ?season wdt:P527 ?episode .          #BGP to retrieve all the episodes of a season
   ?episode wdt:P161 ?castMember.       #BGP to retireve all cast members that participated in the episode
   ?castMember sc:name ?castMemberName.
}
GROUP BY ?castMember ?castMemberName
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q613378'), ('castMemberName', 'Bruno Campos')]
[('castMember', 'http://www.wikidata.org/entity/Q1338682'), ('castMemberName', 'Jeremy Ratchford')]
[('castMember', 'http://www.wikidata.org/entity/Q2601137'), ('castMemberName', 'Elena Satine')]
[('castMember', 'http://www.wikidata.org/entity/Q453774'), ('castMemberName', 'Ellen Geer')]
[('castMember', 'http://www.wikidata.org/entity/Q545288'), ('castMemberName', 'Mimi Kuzyk')]
[('castMember', 'http://www.wikidata.org/entity/Q552031'), ('castMemberName', 'Daniel Roebuck')]
[('castMember', 'http://www.wikidata.org/entity/Q594738'), ('castMemberName', 'Glenn E. Plummer')]
[('castMember', 'http://www.wikidata.org/entity/Q3097804'), ('castMemberName', 'John Bennett Perry')]
[('castMember', 'http://www.wikidata.org/entity/Q171525'), ('castMemberName', 'Jake Johnson')]
[('castMember', 'http://www.wikidata.org/entity/Q3306283'), ('castMemberName', 'Dwight Hicks')]
[('castMember', 'http://ww

In [217]:
# Get properties of 'Cold Case'
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q733960 ?p ?obj .
   ?p sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P5829'), ('name', 'Television Academy Foundation show ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('name', 'number of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1267'), ('name', 'AlloCiné series ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('name', 'title')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P170'), ('name', 'creator')]
[('p', 'http://www.wikidata.org/prop/direct/P1712'), ('name', 'Metacritic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1811'), ('name', 'list of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P2013'), ('name', 'Facebook ID')]
[('p', 'http://www.wiki

In [219]:
# Get instance of 'Cold Case'
queryString = """
SELECT DISTINCT ?superClass ?name
WHERE {
   wd:Q733960 wdt:P31 ?superClass .
   ?superClass sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('superClass', 'http://www.wikidata.org/entity/Q5398426'), ('name', 'television series')]
1


In [220]:
# Get properties of 'television series'
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q5398426 ?p ?obj .
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P9374'), ('name', 'ODLIS ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('name', 'equivalent class')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('name', 'has parts of the class')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P3219'), ('name', 'Encyclopædia Universalis ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), (

In [221]:
# Get properties of 'television series'
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   wd:Q5398426 wdt:P1963 ?obj .
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/P1113'), ('name', 'number of episodes')]
[('obj', 'http://www.wikidata.org/entity/P580'), ('name', 'start time')]
[('obj', 'http://www.wikidata.org/entity/P582'), ('name', 'end time')]
[('obj', 'http://www.wikidata.org/entity/P1476'), ('name', 'title')]
[('obj', 'http://www.wikidata.org/entity/P921'), ('name', 'main subject')]
[('obj', 'http://www.wikidata.org/entity/P527'), ('name', 'has part')]
[('obj', 'http://www.wikidata.org/entity/P495'), ('name', 'country of origin')]
[('obj', 'http://www.wikidata.org/entity/P136'), ('name', 'genre')]
[('obj', 'http://www.wikidata.org/entity/P2437'), ('name', 'number of seasons')]
[('obj', 'http://www.wikidata.org/entity/P449'), ('name', 'original broadcaster')]
[('obj', 'http://www.wikidata.org/entity/P840'), ('name', 'narrative location')]
[('obj', 'http://www.wikidata.org/entity/P915'), ('name', 'filming location')]
[('obj', 'http://www.wikidata.org/entity/P161'), ('name', 'cast member')]
[('ob

In [222]:
# Get the awards received of 'Cold Case'
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   wd:Q733960 wdt:P166 ?obj .
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [224]:
# Get some series tv series
queryString = """
SELECT DISTINCT ?serie ?name
WHERE {
   ?serie wdt:P31 wd:Q5398426 .     #BGP to idenify an instance of 'television series'
   ?serie sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('serie', 'http://www.wikidata.org/entity/Q1154217'), ('name', 'Ironfist Chinmi')]
[('serie', 'http://www.wikidata.org/entity/Q691528'), ('name', 'Wallace and Gromit')]
[('serie', 'http://www.wikidata.org/entity/Q8193809'), ('name', 'Shomuni')]
[('serie', 'http://www.wikidata.org/entity/Q33323'), ('name', 'Everybody Hates Chris')]
[('serie', 'http://www.wikidata.org/entity/Q500769'), ('name', 'Freaks and Geeks')]
[('serie', 'http://www.wikidata.org/entity/Q8350955'), ('name', 'Cordel Encantado')]
[('serie', 'http://www.wikidata.org/entity/Q1145054'), ('name', 'The NBC Mystery Movie')]
[('serie', 'http://www.wikidata.org/entity/Q344081'), ('name', 'Recess')]
[('serie', 'http://www.wikidata.org/entity/Q13647845'), ('name', 'Grijpstra & De Gier')]
[('serie', 'http://www.wikidata.org/entity/Q708661'), ('name', 'Lindenstraße')]
10


In [230]:
# Get some series tv series that received some awards
queryString = """
SELECT DISTINCT ?serie ?name ?awardName
WHERE {
   ?serie wdt:P31 wd:Q5398426 .     #BGP to idenify an instance of 'television series'
   ?serie sc:name ?name.
   ?serie wdt:P166 ?award .
   ?serie sc:name ?awardName.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('serie', 'http://www.wikidata.org/entity/Q691528'), ('name', 'Wallace and Gromit'), ('awardName', 'Wallace and Gromit')]
[('serie', 'http://www.wikidata.org/entity/Q500769'), ('name', 'Freaks and Geeks'), ('awardName', 'Freaks and Geeks')]
[('serie', 'http://www.wikidata.org/entity/Q1145054'), ('name', 'The NBC Mystery Movie'), ('awardName', 'The NBC Mystery Movie')]
[('serie', 'http://www.wikidata.org/entity/Q2484656'), ('name', 'Flikken Maastricht'), ('awardName', 'Flikken Maastricht')]
[('serie', 'http://www.wikidata.org/entity/Q7779266'), ('name', 'Top of the Lake'), ('awardName', 'Top of the Lake')]
[('serie', 'http://www.wikidata.org/entity/Q536110'), ('name', 'Whale Wars'), ('awardName', 'Whale Wars')]
[('serie', 'http://www.wikidata.org/entity/Q607514'), ('name', 'EastEnders'), ('awardName', 'EastEnders')]
[('serie', 'http://www.wikidata.org/entity/Q186219'), ('name', 'True Blood'), ('awardName', 'True Blood')]
[('serie', 'http://www.wikidata.org/entity/Q74801'), ('na

In [231]:
# Get some series tv series that received more than two awards
queryString = """
SELECT DISTINCT ?serie ?name (COUNT(?award) as ?awards)
WHERE {
   ?serie wdt:P31 wd:Q5398426 .     #BGP to idenify an instance of 'television series'
   ?serie sc:name ?name.
   ?serie wdt:P166 ?award .
}
GROUP BY ?serie ?name
HAVING (COUNT(?award) > 2)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('serie', 'http://www.wikidata.org/entity/Q585758'), ('name', 'Boardwalk Empire'), ('awards', '3')]
[('serie', 'http://www.wikidata.org/entity/Q265519'), ('name', 'heute-show'), ('awards', '8')]
[('serie', 'http://www.wikidata.org/entity/Q695478'), ('name', 'Gute Zeiten, schlechte Zeiten'), ('awards', '3')]
[('serie', 'http://www.wikidata.org/entity/Q30599007'), ('name', 'Succession'), ('awards', '8')]
[('serie', 'http://www.wikidata.org/entity/Q1071675'), ('name', 'Road to Avonlea'), ('awards', '5')]
[('serie', 'http://www.wikidata.org/entity/Q2688944'), ('name', 'Polònia'), ('awards', '4')]
[('serie', 'http://www.wikidata.org/entity/Q1136370'), ('name', 'General Hospital'), ('awards', '10')]
[('serie', 'http://www.wikidata.org/entity/Q22998408'), ('name', 'Babylon Berlin'), ('awards', '6')]
[('serie', 'http://www.wikidata.org/entity/Q39049904'), ('name', 'Killing Eve'), ('awards', '9')]
[('serie', 'http://www.wikidata.org/entity/Q56153643'), ('name', 'Watchmen'), ('awards', 

In [27]:
# Get all the tv series in which a cast memebr of 'Cold Case' has acted
queryString = """
SELECT DISTINCT ?serie ?serieName
WHERE {
   ?season wdt:P179 wd:Q733960 .        #BGP to retrieve all the seasons of Cold Case
   ?season wdt:P527 ?episode .          #BGP to retrieve all the episodes of a season
   ?episode wdt:P161 ?castMember.       #BGP to retireve all cast members that participated in the episode
   
   {
       SELECT ?serie ?serieName
       WHERE {
            ?serie wdt:P31 wd:Q5398426 .     #BGP to idenify an instance of 'television series'
            ?serie sc:name ?serieName .
            ?serie wdt:P161 ?castMember .    #BGP to specify that the castMember of 'Cold case' must be part of the cast
       }
   }
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('serie', 'http://www.wikidata.org/entity/Q33323'), ('serieName', 'Everybody Hates Chris')]
[('serie', 'http://www.wikidata.org/entity/Q500769'), ('serieName', 'Freaks and Geeks')]
[('serie', 'http://www.wikidata.org/entity/Q8350955'), ('serieName', 'Cordel Encantado')]
[('serie', 'http://www.wikidata.org/entity/Q1145054'), ('serieName', 'The NBC Mystery Movie')]
[('serie', 'http://www.wikidata.org/entity/Q13647845'), ('serieName', 'Grijpstra & De Gier')]
[('serie', 'http://www.wikidata.org/entity/Q708661'), ('serieName', 'Lindenstraße')]
[('serie', 'http://www.wikidata.org/entity/Q15099450'), ('serieName', 'Prime Minister and I')]
[('serie', 'http://www.wikidata.org/entity/Q15581035'), ('serieName', 'El cielo es para todos')]
[('serie', 'http://www.wikidata.org/entity/Q15737024'), ('serieName', '37 Days')]
[('serie', 'http://www.wikidata.org/entity/Q15933384'), ('serieName', 'Special Affairs Team TEN')]
[('serie', 'http://www.wikidata.org/entity/Q15870780'), ('serieName', 'Go

In [49]:
queryString = """
SELECT ?serie ?serieName (COUNT(?award) as ?awards)
WHERE {

    ?serie wdt:P166 ?award .

    {
        SELECT DISTINCT ?serie ?serieName
        WHERE {
           ?season wdt:P179 wd:Q733960 .        #BGP to retrieve all the seasons of Cold Case
           ?season wdt:P527 ?episode .          #BGP to retrieve all the episodes of a season
           ?episode wdt:P161 ?castMember.       #BGP to retireve all cast members that participated in the episode

           {
               SELECT ?serie ?serieName
               WHERE {
                    ?serie wdt:P31 wd:Q5398426 .     #BGP to idenify an instance of 'television series'
                    ?serie sc:name ?serieName .
                    ?serie wdt:P161 ?castMember .    #BGP to specify that the castMember of 'Cold case' must be part of the cast
               }
           }
        }
    }

    
}
GROUP BY ?serie ?serieName
ORDER BY DESC(?awards)
"""

print("Results")
x=run_query(queryString)

Results
[('serie', 'http://www.wikidata.org/entity/Q23628'), ('serieName', 'The Sopranos'), ('awards', '22')]
[('serie', 'http://www.wikidata.org/entity/Q3577037'), ('serieName', 'The West Wing'), ('awards', '22')]
[('serie', 'http://www.wikidata.org/entity/Q1079'), ('serieName', 'Breaking Bad'), ('awards', '21')]
[('serie', 'http://www.wikidata.org/entity/Q56153643'), ('serieName', 'Watchmen'), ('awards', '20')]
[('serie', 'http://www.wikidata.org/entity/Q192837'), ('serieName', 'Sherlock'), ('awards', '16')]
[('serie', 'http://www.wikidata.org/entity/Q16868648'), ('serieName', 'Fleabag'), ('awards', '12')]
[('serie', 'http://www.wikidata.org/entity/Q1132439'), ('serieName', 'The Practice'), ('awards', '12')]
[('serie', 'http://www.wikidata.org/entity/Q1048856'), ('serieName', 'All in the Family'), ('awards', '11')]
[('serie', 'http://www.wikidata.org/entity/Q16756'), ('serieName', 'Modern Family'), ('awards', '11')]
[('serie', 'http://www.wikidata.org/entity/Q1136370'), ('serieName',

In [51]:
# Get the awards received by 'Grey's anatomy'
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   wd:Q438406 wdt:P166 ?obj .
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q135498'), ('name', 'Genesis Award')]
[('obj', 'http://www.wikidata.org/entity/Q1255198'), ('name', 'Golden Globe Award for Best Television Series – Drama')]
[('obj', 'http://www.wikidata.org/entity/Q1285504'), ('name', 'Primetime Emmy Award for Outstanding Supporting Actress in a Drama Series')]
[('obj', 'http://www.wikidata.org/entity/Q2414873'), ('name', 'Screen Actors Guild Award for Outstanding Performance by a Female Actor in a Drama Series')]
[('obj', 'http://www.wikidata.org/entity/Q3062632'), ('name', 'Primetime Emmy Award for Outstanding Guest Actress in a Drama Series')]
[('obj', 'http://www.wikidata.org/entity/Q2530270'), ('name', 'Screen Actors Guild Award for Outstanding Performance by an Ensemble in a Drama Series')]
[('obj', 'http://www.wikidata.org/entity/Q7243497'), ('name', 'Primetime Emmy Award for Outstanding Casting for a Drama Series')]
[('obj', 'http://www.wikidata.org/entity/Q1259395'), ('name', 'Golden Globe Awa

In [59]:
# Get the awards received by 'Person of Interest'
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   wd:Q564345 wdt:P166 ?obj .
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q6952071'), ('name', 'NAACP Image Award for Outstanding Supporting Actress in a Drama Series')]
1


In [58]:
# Get the awards received by 'Sherlock'
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   wd:Q192837 wdt:P166 ?obj .
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q838121'), ('name', 'Peabody Awards')]
[('obj', 'http://www.wikidata.org/entity/Q7243506'), ('name', 'Primetime Emmy Award for Outstanding Made for Television Movie')]
[('obj', 'http://www.wikidata.org/entity/Q945887'), ('name', 'Primetime Emmy Award for Outstanding Supporting Actor in a Miniseries or a Movie')]
[('obj', 'http://www.wikidata.org/entity/Q989453'), ('name', 'Primetime Emmy Award for Outstanding Lead Actor in a Miniseries or a Movie')]
[('obj', 'http://www.wikidata.org/entity/Q17986500'), ('name', 'Primetime Emmy Award for Outstanding Cinematography for a Miniseries or Movie')]
[('obj', 'http://www.wikidata.org/entity/Q2925700'), ('name', 'British Academy Television Award for Best Supporting Actor')]
[('obj', 'http://www.wikidata.org/entity/Q7243513'), ('name', 'Primetime Emmy Award for Outstanding Special Visual Effects')]
[('obj', 'http://www.wikidata.org/entity/Q7669617'), ('name', 'TCA Award for Outstanding Achievement 

In [1]:
queryString = """
SELECT DISTINCT ?serie ?serieName ?awards
WHERE {
   ?season wdt:P179 wd:Q733960 .        #BGP to retrieve all the seasons of Cold Case
   ?season wdt:P527 ?episode .          #BGP to retrieve all the episodes of a season
   ?episode wdt:P161 ?castMember.       #BGP to retireve all cast members that participated in the episode
   
   {
       SELECT ?serie ?serieName (COUNT(?award) as ?awards)
       WHERE {
            ?serie wdt:P31 wd:Q5398426 .     #BGP to idenify an instance of 'television series'
            ?serie sc:name ?serieName .
            ?serie wdt:P161 ?castMember .    #BGP to specify that the castMember of 'Cold case' must be part of the cast
            ?serie wdt:P166 ?award .
       }
       GROUP BY ?serie ?serieName
       HAVING (COUNT(?award) > 2)
   }
}
"""

print("Results")
x=run_query(queryString)

Results
[('serie', 'http://www.wikidata.org/entity/Q9024286'), ('serieName', 'Los ladrones van a la oficina'), ('awards', '16')]
[('serie', 'http://www.wikidata.org/entity/Q585758'), ('serieName', 'Boardwalk Empire'), ('awards', '69')]
[('serie', 'http://www.wikidata.org/entity/Q265519'), ('serieName', 'heute-show'), ('awards', '136')]
[('serie', 'http://www.wikidata.org/entity/Q207803'), ('serieName', 'Bones'), ('awards', '20')]
[('serie', 'http://www.wikidata.org/entity/Q1077'), ('serieName', 'Star Trek: The Original Series'), ('awards', '24')]
[('serie', 'http://www.wikidata.org/entity/Q464865'), ('serieName', 'The Nanny'), ('awards', '113')]
[('serie', 'http://www.wikidata.org/entity/Q1048363'), ('serieName', 'Party of Five'), ('awards', '11')]
[('serie', 'http://www.wikidata.org/entity/Q564345'), ('serieName', 'Person of Interest'), ('awards', '91')]
[('serie', 'http://www.wikidata.org/entity/Q2685847'), ('serieName', 'Inspector Morse'), ('awards', '3')]
[('serie', 'http://www.wik

In [57]:
queryString = """
SELECT ?serie ?serieName (COUNT(?award) as ?awards)
WHERE {

    ?serie wdt:P166 ?award .

    {
        SELECT DISTINCT ?serie ?serieName
        WHERE {
           ?season wdt:P179 wd:Q733960 .        #BGP to retrieve all the seasons of Cold Case
           ?season wdt:P527 ?episode .          #BGP to retrieve all the episodes of a season
           ?episode wdt:P161 ?castMember.       #BGP to retireve all cast members that participated in the episode

           {
               SELECT ?serie ?serieName
               WHERE {
                    ?serie wdt:P31 wd:Q5398426 .     #BGP to idenify an instance of 'television series'
                    ?serie sc:name ?serieName .
                    ?serie wdt:P161 ?castMember .    #BGP to specify that the castMember of 'Cold case' must be part of the cast
               }
           }
        }
    }

    
}
GROUP BY ?serie ?serieName
HAVING (COUNT(?award) > 2)
ORDER BY DESC(?awards)
"""

print("Results")
x=run_query(queryString)

Results
[('serie', 'http://www.wikidata.org/entity/Q23628'), ('serieName', 'The Sopranos'), ('awards', '22')]
[('serie', 'http://www.wikidata.org/entity/Q3577037'), ('serieName', 'The West Wing'), ('awards', '22')]
[('serie', 'http://www.wikidata.org/entity/Q1079'), ('serieName', 'Breaking Bad'), ('awards', '21')]
[('serie', 'http://www.wikidata.org/entity/Q56153643'), ('serieName', 'Watchmen'), ('awards', '20')]
[('serie', 'http://www.wikidata.org/entity/Q192837'), ('serieName', 'Sherlock'), ('awards', '16')]
[('serie', 'http://www.wikidata.org/entity/Q16868648'), ('serieName', 'Fleabag'), ('awards', '12')]
[('serie', 'http://www.wikidata.org/entity/Q1132439'), ('serieName', 'The Practice'), ('awards', '12')]
[('serie', 'http://www.wikidata.org/entity/Q1048856'), ('serieName', 'All in the Family'), ('awards', '11')]
[('serie', 'http://www.wikidata.org/entity/Q16756'), ('serieName', 'Modern Family'), ('awards', '11')]
[('serie', 'http://www.wikidata.org/entity/Q1136370'), ('serieName',

In [2]:
# Get the awards received by 'Breaking Bad'
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   wd:Q1079 wdt:P166 ?obj .
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q838121'), ('name', 'Peabody Awards')]
[('obj', 'http://www.wikidata.org/entity/Q989439'), ('name', 'Primetime Emmy Award for Outstanding Lead Actor in a Drama Series')]
[('obj', 'http://www.wikidata.org/entity/Q3123491'), ('name', 'Primetime Emmy Award for Outstanding Writing for a Drama Series')]
[('obj', 'http://www.wikidata.org/entity/Q1255198'), ('name', 'Golden Globe Award for Best Television Series – Drama')]
[('obj', 'http://www.wikidata.org/entity/Q8038466'), ('name', 'Writers Guild of America Award for Television: Dramatic Series')]
[('obj', 'http://www.wikidata.org/entity/Q1285504'), ('name', 'Primetime Emmy Award for Outstanding Supporting Actress in a Drama Series')]
[('obj', 'http://www.wikidata.org/entity/Q1286639'), ('name', 'Primetime Emmy Award for Outstanding Supporting Actor in a Drama Series')]
[('obj', 'http://www.wikidata.org/entity/Q3473896'), ('name', 'Satellite Award for Best Supporting Actor – Series, Miniserie

In [3]:
# Get the awards received by Modern Family
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   wd:Q16756 wdt:P166 ?obj .
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1277961'), ('name', 'Primetime Emmy Award for Outstanding Directing for a Comedy Series')]
[('obj', 'http://www.wikidata.org/entity/Q1285970'), ('name', 'Primetime Emmy Award for Outstanding Supporting Actor in a Comedy Series')]
[('obj', 'http://www.wikidata.org/entity/Q596294'), ('name', 'Golden Globe Award for Best Television Series – Musical or Comedy')]
[('obj', 'http://www.wikidata.org/entity/Q3045762'), ('name', 'Primetime Emmy Award for Outstanding Writing for a Comedy Series')]
[('obj', 'http://www.wikidata.org/entity/Q7669610'), ('name', 'TCA Award for Individual Achievement in Comedy')]
[('obj', 'http://www.wikidata.org/entity/Q989450'), ('name', 'Primetime Emmy Award for Outstanding Supporting Actress in a Comedy Series')]
[('obj', 'http://www.wikidata.org/entity/Q2110156'), ('name', 'Primetime Emmy Award for Outstanding Comedy Series')]
[('obj', 'http://www.wikidata.org/entity/Q7243495'), ('name', 'Primetime Emmy Award for 

Final query for this task

In [4]:
queryString = """
SELECT ?serie ?serieName (COUNT(?award) as ?awards)
WHERE {

    ?serie wdt:P166 ?award .

    {
        SELECT DISTINCT ?serie ?serieName
        WHERE {
           ?season wdt:P179 wd:Q733960 .        #BGP to retrieve all the seasons of Cold Case
           ?season wdt:P527 ?episode .          #BGP to retrieve all the episodes of a season
           ?episode wdt:P161 ?castMember.       #BGP to retireve all cast members that participated in the episode

           {
               SELECT ?serie ?serieName
               WHERE {
                    ?serie wdt:P31 wd:Q5398426 .     #BGP to idenify an instance of 'television series'
                    ?serie sc:name ?serieName .
                    ?serie wdt:P161 ?castMember .    #BGP to specify that the castMember of 'Cold case' must be part of the cast
               }
           }
        }
    }

    
}
GROUP BY ?serie ?serieName
HAVING (COUNT(?award) > 2)
ORDER BY DESC(?awards)
"""

print("Results")
x=run_query(queryString)

Results
[('serie', 'http://www.wikidata.org/entity/Q23628'), ('serieName', 'The Sopranos'), ('awards', '22')]
[('serie', 'http://www.wikidata.org/entity/Q3577037'), ('serieName', 'The West Wing'), ('awards', '22')]
[('serie', 'http://www.wikidata.org/entity/Q1079'), ('serieName', 'Breaking Bad'), ('awards', '21')]
[('serie', 'http://www.wikidata.org/entity/Q56153643'), ('serieName', 'Watchmen'), ('awards', '20')]
[('serie', 'http://www.wikidata.org/entity/Q192837'), ('serieName', 'Sherlock'), ('awards', '16')]
[('serie', 'http://www.wikidata.org/entity/Q16868648'), ('serieName', 'Fleabag'), ('awards', '12')]
[('serie', 'http://www.wikidata.org/entity/Q1132439'), ('serieName', 'The Practice'), ('awards', '12')]
[('serie', 'http://www.wikidata.org/entity/Q1048856'), ('serieName', 'All in the Family'), ('awards', '11')]
[('serie', 'http://www.wikidata.org/entity/Q16756'), ('serieName', 'Modern Family'), ('awards', '11')]
[('serie', 'http://www.wikidata.org/entity/Q1136370'), ('serieName',